A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [ ]:
classdef CDCMSSpreadDualRangeAccrualInverseFloater < Security
    %RangeAccrual is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        nominal
        coupon
        couponFreq
        tenor
        
        lower
        upper
        
        tenorL
        tenorS
        lowerS
        upperS
        rangeDays
        callYN
        schedule
        couponSchedule
        callSchedule
        manualScheduleYN
        hedgeBuffer
        
        couponLowerBound
    end
    
    methods
        % constructor
        function rangeAccrual = CDCMSSpreadDualRangeAccrualInverseFloater(params)
                rangeAccrual.payCcy = params.payCcy;
                rangeAccrual.startDate = params.startDate;
                rangeAccrual.maturity = params.maturity;
                
                %rangeAccrual.maturityDate = params.startDate.AddDate(params.maturity,'year');
                rangeAccrual.nominal = params.params('nominal');
                rangeAccrual.coupon = params.params('coupon');
                rangeAccrual.couponFreq= params.params('couponFreq');
                rangeAccrual.tenor = params.params('tenor');
                rangeAccrual.tenorL = params.params('tenorL');
                rangeAccrual.tenorS = params.params('tenorS');
                rangeAccrual.lower = params.params('lower');
                rangeAccrual.upper = params.params('upper');
                rangeAccrual.lowerS = params.params('lowerS');
                rangeAccrual.upperS = params.params('upperS');
                rangeAccrual.rangeDays = params.params('rangeDays');
                rangeAccrual.callYN = params.params('callYN');
                rangeAccrual.manualScheduleYN = params.params('manualScheduleYN');
                if rangeAccrual.manualScheduleYN
                    rangeAccrual.couponSchedule = rangeAccrual.ManualScheduleGenerate(params.params('couponScheduleInt'));
                    rangeAccrual.callSchedule = rangeAccrual.ManualScheduleGenerate(params.params('callScheduleInt'));
                    
                
                else
                    maturityDate = params.startDate.AddDate(params.maturity,'year');
                    rangeAccrual.schedule = rangeAccrual.ScheduleGenerate(rangeAccrual.startDate,maturityDate,rangeAccrual.couponFreq);
                
                end
                
                if isKey(params.params,'hedgeBuffer')
                    rangeAccrual.hedgeBuffer = params.params('hedgeBuffer');
                else
                    rangeAccrual.hedgeBuffer = 0.0;
                end
                
                rangeAccrual.couponLowerBound = params.params('couponLowerBound');
                 
        end
        
        function out = ManualScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.numOfPeriod = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.startDates(i) = H_Date(scheduleInt(i,2));
                out.endDates(i) = H_Date(scheduleInt(i,3));
                out.payDates(i) = H_Date(scheduleInt(i,4));
                out.dayCount(i) = out.endDates(i).DateDiff(out.startDates(i));
                out.dayCountFraction(i) = out.dayCount(i)/ 365.0;
            end
            
            
        end
        
        function out = ScheduleGenerate(rangeAccrual,initialDate,maturityDate,freq)
            schedule = ScheduleGenerate(initialDate,maturityDate,freq);
            out.startDates= schedule.startDates;
            out.resetDates = schedule.resetDates;
            out.endDates = schedule.endDates;
            out.payDates = schedule.payDates;
            out.dayCount = schedule.dayCount;
            out.dayCountFraction = schedule.dayCountFraction;
            out.numOfPeriod = schedule.numOfPeriod;
        end
        
        function out = numMethodTimeStepsGenerate(rangeAccrual,valueDate,schedule,mcOneTimeStep)
            
            periodCount = schedule.numOfPeriod;
            out.timeStepsPerPeriod =  cell(periodCount,1);
            
            out.startTimeIdx = zeros(periodCount,1);
            out.endM1TimeIdx = zeros(periodCount,1);
            
            out.numOfTimeSteps = zeros(periodCount,1);
            out.totalTimeSteps = [];
            out.totalTimeStepsSize = 0;
            numOfAlivePeriod = 0;
            
            for i=1:periodCount
                timeSteps = [];
                if DateDiff(valueDate,schedule.payDates(i)) >= 0
                    out.timeStepsPerPeriod(i) = timeSteps;
                    continue;
                end
                
                numOfAlivePeriod = numOfAlivePeriod + 1;
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    startDate = valueDate;
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    %out.timeStepsPerPeriod {i,1}.timeSteps = [];
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = 1;
                    out.endM1TimeIdx(i) = length(timeSteps);
                    
                else
                    startDate = schedule.startDates(i);
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = out.endM1TimeIdx(i-1) + 1;
                    out.endM1TimeIdx(i) = out.endM1TimeIdx(i-1) + length(timeSteps);
                    
                end
            end
            
            % endDate of the last period (= Security's maturityDate should
            % be included
            
            endTime = DateDiff(schedule.endDates(periodCount),valueDate);
            out.totalTimeSteps = [out.totalTimeSteps;endTime];
            out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
            out.totalTimeStepsSize = length(out.totalTimeSteps);
            
            out.periodCount = periodCount;
            out.numOfAlivePeriod = numOfAlivePeriod;
        end
        
        function out = generateModelStates(rangeAccrual,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                sqrtDt = sqrt((nextStep-currentStep)/365.0);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                L = chol(stateLocalVariance);
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+2,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+2,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+2,:) = prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = computePriceMCForward(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            
            % numMethodTimeSteps Generate
            mcOneTimeStep = 1;
            outInfo = rangeAccrual.numMethodTimeStepsGenerate(valueDate,schedule,mcOneTimeStep);
            
            numMethodInfo.timeStepsInfo = outInfo;
            
            % mc random number init
            %pathSize = 2^16;
            pathSize = 5000;
            
            %pathSize = 2^10;
            
            rng('default');
            rng(0);
            
            numOfFactors = 2;
            
            Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
            Z=[Z,-Z];
            
            numMethodInfo.pathSize = pathSize;
            numMethodInfo.numOfFactors = numOfFactors;
            numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
            numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
            numMethodInfo.Z = Z;
            
            % numeraire info
            numMatTime = numMethodInfo.totalTimeSteps(end);
            dfNumMat = model.DF(numMatTime/365.0);
            
            %generate model states for the given timeSteps
            modelStates = rangeAccrual.generateModelStates(valueDate,schedule,numMethodInfo,model);
            
            % induct backward and calculate cashflow
            
            
            periodCount = schedule.numOfPeriod;
            
            nonCall.npv = 0;
            
            nonCall.cashflow_e = zeros(periodCount + 1,1);
            nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
            nonCall.rangeProb_e = zeros(periodCount + 1,1);
            
            nonCall.df_e = zeros(periodCount + 1,1);
            nonCall.df_c = zeros(periodCount + 1,1);
            
            % sum of cashflow_npv
            nonCall.payoff = zeros(1,pathSize);
            
            nonCall.payoffStateVectors_cashflow = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVectors_cashflow_npv = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVector_df = zeros(periodCount + 1,pathSize);
            
            payoffStateVectors_cashflow = zeros(1, pathSize);
            payoffStateVectors_cashflow_npv = zeros(1, pathSize);
            payoffStateVectors_df = zeros(1,pathSize);
            
            % we calculate cashflow backward from maturityDate to valueDate
            % rangeAccrual coupons are added at the startDate of the Period
            % At maturity
            
            nominal = rangeAccrual.nominal;
            coupon = rangeAccrual.coupon;
            
            currentTime = numMethodInfo.totalTimeSteps(end);
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            currentTimeIdx = totalTimeStepsSize;
            
            modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
            payoffStateVectors_cashflow = nominal*ones(1,pathSize);
            nonCall.payoffStateVectors_cashflow(end,:) = payoffStateVectors_cashflow;
            
            payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
            nonCall.payoffStateVectors_cashflow_npv(end,:) = payoffStateVectors_cashflow_npv;
            
            nonCall.cashflow_npv_e(end) = mean(payoffStateVectors_cashflow_npv);
            
            nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
            
            payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
            
            nonCall.payoffStateVector_df(end,:) = payoffStateVectors_df;
            nonCall.df_e(end) = mean(nonCall.payoffStateVector_df(end,:));
            nonCall.df_c(end) = model.DF(currentTime/365.0);
%             
%             nonCall.cashflow_npv_df(end,:) =  nonCall.npv/nominal;
%             nonCall.cashflow_npv_df_e(end) = mean(nonCall.cashflow_npv_df(end,:));
            
            lastTimeIdx = currentTimeIdx;
            totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
            
            for i=periodCount:-1:1
                % past schedule neglect
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                endM1TimeIdx = numMethodInfo.timeStepsInfo.endM1TimeIdx(i);
                
                %% FromToInduction start
                
                %modelsStates induct backward(lastTimeIdx to endM1TimeIdx)
                while endM1TimeIdx < lastTimeIdx
                    lastTimeIdx = lastTimeIdx -1;
                    if lastTimeIdx == endM1TimeIdx break;end
                end
                
                currentTimeIdx = lastTimeIdx;
                currentTime = totalTimeSteps(currentTimeIdx);
                % now we are at endM1TimeIdx
                % variables for coupon valuation initialized
                ralower = rangeAccrual.lower;
                raupper = rangeAccrual.upper;
                
                ralowerS = rangeAccrual.lowerS;
                raupperS = rangeAccrual.upperS;
                
                tenorCMS = rangeAccrual.tenor;
                
                tenorLong = rangeAccrual.tenorL;
                tenorShort = rangeAccrual.tenorS;
                
                cD = 0; % cumulated day count fraction
                cP = 0; 
                dI = 0; % dummy idx for loop
                dcfC = 0;
                dcfN = 0;
                dD = 0;
                idxCMS = zeros(1,pathSize);
                
                idxL = zeros(1,pathSize);
                idxS = zeros(1,pathSize);
                
                idxC = zeros(1,pathSize);
                idxN = zeros(1,pathSize);
                
                spreadC = zeros(1,pathSize);
                spreadN = zeros(1,pathSize);
                probSpread = zeros(1,pathSize);
                simpleYN = true;
                numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                
                for j=numOfTimeSteps:-1:1
                    % evaluate dealdescription
                    % last step is omitted
                    currentTime = totalTimeSteps(currentTimeIdx);
                    dcfC = currentTime/365.0;
                    if j~= numOfTimeSteps
                        dD = (dcfN-dcfC);
                    end
                    
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                    idxCMS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                    idxC = idxCMS;
                    
                    idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    spreadC = idxL - idxS;
                     
%                     probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                    probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                    
                    cD = cD + dD;
                    cP = cP + probSpread;
  
                    dcfN = dcfC;
                    
                    idxN = idxC;
                    spreadN = spreadC;
                    
                    %induct backward to the next time step
                    % one time backward induction
                    lastTimeIdx = currentTimeIdx;
                    
                    if startTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        currentTimeIdx = lastTimeIdx;
                    end
                    
                    
                end
                
                %% FromToInduction end
                %% AtDate operation
                % Adding a new cashflow to the payoff
                
                dcf = schedule.dayCountFraction(i);
                payoffStateVectors_cashflow = nominal * coupon * dcf * cP/cD;
                
                % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                % i.e. currentTimeIdx has not changed
                % no need to upodate modelStateT
                % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                % since cashflow's are added at the startDate(i) we need to
                % multiply cashflow with the df(startDate(i), payDate(i))
                payTime = DateDiff(schedule.payDates(i),valueDate);
                df_se = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);
                
                for k=1:pathSize
                    payoffStateVectors_cashflow(k) = payoffStateVectors_cashflow(k) + df_se(k);
                end
                
                nonCall.payoffStateVectors_cashflow(i,:) = payoffStateVectors_cashflow;
                
                nonCall.cashflow_e(i) = mean(payoffStateVectors_cashflow);
                nonCall.rangeProb_e(i) = mean(nonCall.cashflow_e(i)/(nominal * coupon * dcf));
                
                payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
                nonCall.cashflow_npv_e(i) = mean(payoffStateVectors_cashflow_npv);
                
                nonCall.payoffStateVectors_cashflow_npv(i,:) = payoffStateVectors_cashflow_npv;
                nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
                
                payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
                
                nonCall.payoffStateVector_df(i,:) = payoffStateVectors_df;
                nonCall.df_e(i) = mean(nonCall.payoffStateVector_df(i,:));
                nonCall.df_c(i) = model.DF(currentTime/365.0);
                
            end
            
            nonCall.npv = mean(nonCall.payoff);
  
            out.cashflow= nonCall.cashflow_e;
            out.cashflowNpv = nonCall.cashflow_npv_e;
            out.df_e = nonCall.df_e;
            out.df_c = nonCall.df_c;
            out.daycountFraction = schedule.dayCountFraction(i);
            out.rangeProb = nonCall.rangeProb_e;
            out.npv = nonCall.npv;
           
        end
        
        function out = computePrice(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            periodCount = schedule.numOfPeriod;
            rangeProb = zeros(periodCount,1);
            discountFactor = zeros(periodCount,1);
            daycountFraction = zeros(periodCount,1);
            cashflow = zeros(periodCount,1);
            cashflowNpv = zeros(periodCount,1);
            rangeDays = rangeAccrual.rangeDays;
            npv = 0;
            for i=1:periodCount
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end;
                
                observeDate = schedule.startDates(i);
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    observeDate = valueDate;
                    observeDate = AddDate(observeDate,1,'day');
                else
                    observeDate = schedule.startDates(i);
                end
                
                observeDays = DateDiff(schedule.endDates(i),schedule.startDates(i));
                days = DateDiff(schedule.endDates(i), observeDate);
                discountFactor(i) = model.DF( DateDiff(schedule.payDates(i),valueDate)/365.0);
                daycountFraction(i) = schedule.dayCountFraction(i);
                for j=1:days
                    toDate = observeDate;
                    tpDate = schedule.payDates(i);
                    tenor1 = rangeAccrual.tenorL;
                    tenor2 = rangeAccrual.tenorS;
                    tau =0.25;
                    lower = rangeAccrual.lower;
                    upper = rangeAccrual.upper;
                    rangeValue = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper);
%                     rangeValue = 1.0;
                    rangeProb(i) = rangeProb(i) + rangeValue;
                    observeDate = AddDate(observeDate,1,'day');
                end
                cashflow(i) = rangeAccrual.nominal * rangeAccrual.coupon * (rangeProb(i) + rangeDays(i))/observeDays * daycountFraction(i);
                cashflowNpv(i) = cashflow(i) * discountFactor(i);
                npv = npv + cashflowNpv(i);
                % bug fixed observeDays -> days
                rangeProb(i) = rangeProb(i)/days;
                
                if i == periodCount
                    npv = npv + rangeAccrual.nominal * discountFactor(i);
                end
            end
            out.cashflow= cashflow;
            out.cashflowNpv = cashflowNpv;
            out.discountFactor = discountFactor;
            out.daycountFraction = daycountFraction;
            out.rangeProb = rangeProb;
            out.npv = npv;
        end
        
%         function out = discountPayoff(rangeAccrual,eventTime,numMatTime ...
%                 ,model,modelStatesT,cashflow)
%             
%             dfNumMat = model.DF(numMatTime/365.0);
%             % for discountPayoff T=numMatTime
%             % P(0,TnumMat)/P(Te,Tnummat)*Payoff
%             %
%             discountT = model.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
%             stateSize = size(modelStatesT,2);
%             payoff = zeros(1,stateSize);
%             for i=1:stateSize
%                 payoff(i)= dfNumMat/discountT(i)*cashflow(i);
%             end    
%             out = payoff;
%         end
        
        
        
    end
    
end



In [ ]:
classdef CDCMSSpreadDualRangeAccrual < Security
    %RangeAccrual is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        nominal
        coupon
        couponFreq
        tenor
        
        lower
        upper
        
        tenorL
        tenorS
        lowerS
        upperS
        rangeDays
        callYN
        schedule
        couponSchedule
        callSchedule
        manualScheduleYN
        hedgeBuffer
    end
    
    methods
        % constructor
        function rangeAccrual = CDCMSSpreadDualRangeAccrual(params)
                rangeAccrual.payCcy = params.payCcy;
                rangeAccrual.startDate = params.startDate;
                rangeAccrual.maturity = params.maturity;
                
                %rangeAccrual.maturityDate = params.startDate.AddDate(params.maturity,'year');
                rangeAccrual.nominal = params.params('nominal');
                rangeAccrual.coupon = params.params('coupon');
                rangeAccrual.couponFreq= params.params('couponFreq');
                rangeAccrual.tenor = params.params('tenor');
                rangeAccrual.tenorL = params.params('tenorL');
                rangeAccrual.tenorS = params.params('tenorS');
                rangeAccrual.lower = params.params('lower');
                rangeAccrual.upper = params.params('upper');
                rangeAccrual.lowerS = params.params('lowerS');
                rangeAccrual.upperS = params.params('upperS');
                rangeAccrual.rangeDays = params.params('rangeDays');
                rangeAccrual.callYN = params.params('callYN');
                rangeAccrual.manualScheduleYN = params.params('manualScheduleYN');
                if rangeAccrual.manualScheduleYN
                    rangeAccrual.couponSchedule = rangeAccrual.ManualScheduleGenerate(params.params('couponScheduleInt'));
                    rangeAccrual.callSchedule = rangeAccrual.ManualScheduleGenerate(params.params('callScheduleInt'));
                    
                
                else
                    maturityDate = params.startDate.AddDate(params.maturity,'year');
                    rangeAccrual.schedule = rangeAccrual.ScheduleGenerate(rangeAccrual.startDate,maturityDate,rangeAccrual.couponFreq);
                
                end
                
                if isKey(params.params,'hedgeBuffer')
                    rangeAccrual.hedgeBuffer = params.params('hedgeBuffer');
                else
                    rangeAccrual.hedgeBuffer = 0.0;
                end
                 
        end
        
        function out = ManualScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.numOfPeriod = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.startDates(i) = H_Date(scheduleInt(i,2));
                out.endDates(i) = H_Date(scheduleInt(i,3));
                out.payDates(i) = H_Date(scheduleInt(i,4));
                out.dayCount(i) = out.endDates(i).DateDiff(out.startDates(i));
                out.dayCountFraction(i) = out.dayCount(i)/ 365.0;
            end
            
            
        end
        
        function out = ScheduleGenerate(rangeAccrual,initialDate,maturityDate,freq)
            schedule = ScheduleGenerate(initialDate,maturityDate,freq);
            out.startDates= schedule.startDates;
            out.resetDates = schedule.resetDates;
            out.endDates = schedule.endDates;
            out.payDates = schedule.payDates;
            out.dayCount = schedule.dayCount;
            out.dayCountFraction = schedule.dayCountFraction;
            out.numOfPeriod = schedule.numOfPeriod;
        end
        
        function out = numMethodTimeStepsGenerate(rangeAccrual,valueDate,schedule,mcOneTimeStep)
            
            periodCount = schedule.numOfPeriod;
            out.timeStepsPerPeriod =  cell(periodCount,1);
            
            out.startTimeIdx = zeros(periodCount,1);
            out.endM1TimeIdx = zeros(periodCount,1);
            
            out.numOfTimeSteps = zeros(periodCount,1);
            out.totalTimeSteps = [];
            out.totalTimeStepsSize = 0;
            numOfAlivePeriod = 0;
            
            for i=1:periodCount
                timeSteps = [];
                if DateDiff(valueDate,schedule.payDates(i)) >= 0
                    out.timeStepsPerPeriod(i) = timeSteps;
                    continue;
                end
                
                numOfAlivePeriod = numOfAlivePeriod + 1;
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    startDate = valueDate;
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    %out.timeStepsPerPeriod {i,1}.timeSteps = [];
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = 1;
                    out.endM1TimeIdx(i) = length(timeSteps);
                    
                else
                    startDate = schedule.startDates(i);
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = out.endM1TimeIdx(i-1) + 1;
                    out.endM1TimeIdx(i) = out.endM1TimeIdx(i-1) + length(timeSteps);
                    
                end
            end
            
            % endDate of the last period (= Security's maturityDate should
            % be included
            
            endTime = DateDiff(schedule.endDates(periodCount),valueDate);
            out.totalTimeSteps = [out.totalTimeSteps;endTime];
            out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
            out.totalTimeStepsSize = length(out.totalTimeSteps);
            
            out.periodCount = periodCount;
            out.numOfAlivePeriod = numOfAlivePeriod;
        end
        
        function out = generateModelStates(rangeAccrual,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                sqrtDt = sqrt((nextStep-currentStep)/365.0);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                L = chol(stateLocalVariance);
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+2,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+2,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+2,:) = prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = computePriceMCForward(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            
            % numMethodTimeSteps Generate
            mcOneTimeStep = 1;
            outInfo = rangeAccrual.numMethodTimeStepsGenerate(valueDate,schedule,mcOneTimeStep);
            
            numMethodInfo.timeStepsInfo = outInfo;
            
            % mc random number init
            %pathSize = 2^16;
            pathSize = 5000;
            
            %pathSize = 2^10;
            
            rng('default');
            rng(0);
            
            numOfFactors = 2;
            
            Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
            Z=[Z,-Z];
            
            numMethodInfo.pathSize = pathSize;
            numMethodInfo.numOfFactors = numOfFactors;
            numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
            numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
            numMethodInfo.Z = Z;
            
            % numeraire info
            numMatTime = numMethodInfo.totalTimeSteps(end);
            dfNumMat = model.DF(numMatTime/365.0);
            
            %generate model states for the given timeSteps
            modelStates = rangeAccrual.generateModelStates(valueDate,schedule,numMethodInfo,model);
            
            % induct backward and calculate cashflow
            
            
            periodCount = schedule.numOfPeriod;
            
            nonCall.npv = 0;
            
            nonCall.cashflow_e = zeros(periodCount + 1,1);
            nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
            nonCall.rangeProb_e = zeros(periodCount + 1,1);
            
            nonCall.df_e = zeros(periodCount + 1,1);
            nonCall.df_c = zeros(periodCount + 1,1);
            
            % sum of cashflow_npv
            nonCall.payoff = zeros(1,pathSize);
            
            nonCall.payoffStateVectors_cashflow = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVectors_cashflow_npv = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVector_df = zeros(periodCount + 1,pathSize);
            
            payoffStateVectors_cashflow = zeros(1, pathSize);
            payoffStateVectors_cashflow_npv = zeros(1, pathSize);
            payoffStateVectors_df = zeros(1,pathSize);
            
            % we calculate cashflow backward from maturityDate to valueDate
            % rangeAccrual coupons are added at the startDate of the Period
            % At maturity
            
            nominal = rangeAccrual.nominal;
            coupon = rangeAccrual.coupon;
            
            currentTime = numMethodInfo.totalTimeSteps(end);
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            currentTimeIdx = totalTimeStepsSize;
            
            modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
            payoffStateVectors_cashflow = nominal*ones(1,pathSize);
            nonCall.payoffStateVectors_cashflow(end,:) = payoffStateVectors_cashflow;
            
            payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
            nonCall.payoffStateVectors_cashflow_npv(end,:) = payoffStateVectors_cashflow_npv;
            
            nonCall.cashflow_npv_e(end) = mean(payoffStateVectors_cashflow_npv);
            
            nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
            
            payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
            
            nonCall.payoffStateVector_df(end,:) = payoffStateVectors_df;
            nonCall.df_e(end) = mean(nonCall.payoffStateVector_df(end,:));
            nonCall.df_c(end) = model.DF(currentTime/365.0);
%             
%             nonCall.cashflow_npv_df(end,:) =  nonCall.npv/nominal;
%             nonCall.cashflow_npv_df_e(end) = mean(nonCall.cashflow_npv_df(end,:));
            
            lastTimeIdx = currentTimeIdx;
            totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
            
            for i=periodCount:-1:1
                % past schedule neglect
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                endM1TimeIdx = numMethodInfo.timeStepsInfo.endM1TimeIdx(i);
                
                %% FromToInduction start
                
                %modelsStates induct backward(lastTimeIdx to endM1TimeIdx)
                while endM1TimeIdx < lastTimeIdx
                    lastTimeIdx = lastTimeIdx -1;
                    if lastTimeIdx == endM1TimeIdx break;end
                end
                
                currentTimeIdx = lastTimeIdx;
                currentTime = totalTimeSteps(currentTimeIdx);
                % now we are at endM1TimeIdx
                % variables for coupon valuation initialized
                ralower = rangeAccrual.lower;
                raupper = rangeAccrual.upper;
                
                ralowerS = rangeAccrual.lowerS;
                raupperS = rangeAccrual.upperS;
                
                tenorCMS = rangeAccrual.tenor;
                
                tenorLong = rangeAccrual.tenorL;
                tenorShort = rangeAccrual.tenorS;
                
                cD = 0; % cumulated day count fraction
                cP = 0; 
                dI = 0; % dummy idx for loop
                dcfC = 0;
                dcfN = 0;
                dD = 0;
                idxCMS = zeros(1,pathSize);
                
                idxL = zeros(1,pathSize);
                idxS = zeros(1,pathSize);
                
                idxC = zeros(1,pathSize);
                idxN = zeros(1,pathSize);
                
                spreadC = zeros(1,pathSize);
                spreadN = zeros(1,pathSize);
                probSpread = zeros(1,pathSize);
                simpleYN = true;
                numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                
                for j=numOfTimeSteps:-1:1
                    % evaluate dealdescription
                    % last step is omitted
                    currentTime = totalTimeSteps(currentTimeIdx);
                    dcfC = currentTime/365.0;
                    if j~= numOfTimeSteps
                        dD = (dcfN-dcfC);
                    end
                    
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                    idxCMS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                    idxC = idxCMS;
                    
                    idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    spreadC = idxL - idxS;
                     
%                     probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                    probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                    
                    cD = cD + dD;
                    cP = cP + probSpread;
  
                    dcfN = dcfC;
                    
                    idxN = idxC;
                    spreadN = spreadC;
                    
                    %induct backward to the next time step
                    % one time backward induction
                    lastTimeIdx = currentTimeIdx;
                    
                    if startTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        currentTimeIdx = lastTimeIdx;
                    end
                    
                    
                end
                
                %% FromToInduction end
                %% AtDate operation
                % Adding a new cashflow to the payoff
                
                dcf = schedule.dayCountFraction(i);
                payoffStateVectors_cashflow = nominal * coupon * dcf * cP/cD;
                
                % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                % i.e. currentTimeIdx has not changed
                % no need to upodate modelStateT
                % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                % since cashflow's are added at the startDate(i) we need to
                % multiply cashflow with the df(startDate(i), payDate(i))
                payTime = DateDiff(schedule.payDates(i),valueDate);
                df_se = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);
                
                for k=1:pathSize
                    payoffStateVectors_cashflow(k) = payoffStateVectors_cashflow(k) + df_se(k);
                end
                
                nonCall.payoffStateVectors_cashflow(i,:) = payoffStateVectors_cashflow;
                
                nonCall.cashflow_e(i) = mean(payoffStateVectors_cashflow);
                nonCall.rangeProb_e(i) = mean(nonCall.cashflow_e(i)/(nominal * coupon * dcf));
                
                payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
                nonCall.cashflow_npv_e(i) = mean(payoffStateVectors_cashflow_npv);
                
                nonCall.payoffStateVectors_cashflow_npv(i,:) = payoffStateVectors_cashflow_npv;
                nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
                
                payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
                
                nonCall.payoffStateVector_df(i,:) = payoffStateVectors_df;
                nonCall.df_e(i) = mean(nonCall.payoffStateVector_df(i,:));
                nonCall.df_c(i) = model.DF(currentTime/365.0);
                
            end
            
            nonCall.npv = mean(nonCall.payoff);
  
            out.cashflow= nonCall.cashflow_e;
            out.cashflowNpv = nonCall.cashflow_npv_e;
            out.df_e = nonCall.df_e;
            out.df_c = nonCall.df_c;
            out.daycountFraction = schedule.dayCountFraction(i);
            out.rangeProb = nonCall.rangeProb_e;
            out.npv = nonCall.npv;
           
        end
        
        function out = computePrice(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            periodCount = schedule.numOfPeriod;
            rangeProb = zeros(periodCount,1);
            discountFactor = zeros(periodCount,1);
            daycountFraction = zeros(periodCount,1);
            cashflow = zeros(periodCount,1);
            cashflowNpv = zeros(periodCount,1);
            rangeDays = rangeAccrual.rangeDays;
            npv = 0;
            for i=1:periodCount
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end;
                
                observeDate = schedule.startDates(i);
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    observeDate = valueDate;
                    observeDate = AddDate(observeDate,1,'day');
                else
                    observeDate = schedule.startDates(i);
                end
                
                observeDays = DateDiff(schedule.endDates(i),schedule.startDates(i));
                days = DateDiff(schedule.endDates(i), observeDate);
                discountFactor(i) = model.DF( DateDiff(schedule.payDates(i),valueDate)/365.0);
                daycountFraction(i) = schedule.dayCountFraction(i);
                for j=1:days
                    toDate = observeDate;
                    tpDate = schedule.payDates(i);
                    tenor1 = rangeAccrual.tenorL;
                    tenor2 = rangeAccrual.tenorS;
                    tau =0.25;
                    lower = rangeAccrual.lower;
                    upper = rangeAccrual.upper;
                    rangeValue = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper);
%                     rangeValue = 1.0;
                    rangeProb(i) = rangeProb(i) + rangeValue;
                    observeDate = AddDate(observeDate,1,'day');
                end
                cashflow(i) = rangeAccrual.nominal * rangeAccrual.coupon * (rangeProb(i) + rangeDays(i))/observeDays * daycountFraction(i);
                cashflowNpv(i) = cashflow(i) * discountFactor(i);
                npv = npv + cashflowNpv(i);
                % bug fixed observeDays -> days
                rangeProb(i) = rangeProb(i)/days;
                
                if i == periodCount
                    npv = npv + rangeAccrual.nominal * discountFactor(i);
                end
            end
            out.cashflow= cashflow;
            out.cashflowNpv = cashflowNpv;
            out.discountFactor = discountFactor;
            out.daycountFraction = daycountFraction;
            out.rangeProb = rangeProb;
            out.npv = npv;
        end
        
%         function out = discountPayoff(rangeAccrual,eventTime,numMatTime ...
%                 ,model,modelStatesT,cashflow)
%             
%             dfNumMat = model.DF(numMatTime/365.0);
%             % for discountPayoff T=numMatTime
%             % P(0,TnumMat)/P(Te,Tnummat)*Payoff
%             %
%             discountT = model.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
%             stateSize = size(modelStatesT,2);
%             payoff = zeros(1,stateSize);
%             for i=1:stateSize
%                 payoff(i)= dfNumMat/discountT(i)*cashflow(i);
%             end    
%             out = payoff;
%         end
        
        
        
    end
    
end



In [ ]:
classdef CDAndCDKTBSpreadDualRA < Security
    %RangeAccrual is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        nominal
        couponInfo
        callInfo
        manualScheduleYN
        
    end
    
    methods
        % constructor
        function rangeAccrual = CDAndCDKTBSpreadDualRA(params)
                rangeAccrual.payCcy = params.payCcy;
                rangeAccrual.startDate = params.startDate;
                rangeAccrual.maturity = params.maturity;
                
                %rangeAccrual.maturityDate = params.startDate.AddDate(params.maturity,'year');
                rangeAccrual.nominal = params.params('nominal');
                rangeAccrual.coupon = params.params('coupon');
                rangeAccrual.couponFreq= params.params('couponFreq');
                rangeAccrual.tenor = params.params('tenor');
                rangeAccrual.tenorL = params.params('tenorL');
                rangeAccrual.tenorS = params.params('tenorS');
                rangeAccrual.lower = params.params('lower');
                rangeAccrual.upper = params.params('upper');
                rangeAccrual.lowerS = params.params('lowerS');
                rangeAccrual.upperS = params.params('upperS');
                rangeAccrual.rangeDays = params.params('rangeDays');
                rangeAccrual.callYN = params.params('callYN');
                rangeAccrual.manualScheduleYN = params.params('manualScheduleYN');
                if rangeAccrual.manualScheduleYN
                    rangeAccrual.couponSchedule = rangeAccrual.ManualScheduleGenerate(params.params('couponScheduleInt'));
                    rangeAccrual.callSchedule = rangeAccrual.ManualScheduleGenerate(params.params('callScheduleInt'));
                    
                
                else
                    maturityDate = params.startDate.AddDate(params.maturity,'year');
                    rangeAccrual.schedule = rangeAccrual.ScheduleGenerate(rangeAccrual.startDate,maturityDate,rangeAccrual.couponFreq);
                
                end
                 
        end
        
        function out = ManualScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.numOfPeriod = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.startDates(i) = H_Date(scheduleInt(i,2));
                out.endDates(i) = H_Date(scheduleInt(i,3));
                out.payDates(i) = H_Date(scheduleInt(i,4));
                out.dayCount(i) = out.endDates(i).DateDiff(out.startDates(i));
                out.dayCountFraction(i) = out.dayCount(i)/ 365.0;
            end
            
            
        end
        
        function out = ScheduleGenerate(rangeAccrual,initialDate,maturityDate,freq)
            schedule = ScheduleGenerate(initialDate,maturityDate,freq);
            out.startDates= schedule.startDates;
            out.resetDates = schedule.resetDates;
            out.endDates = schedule.endDates;
            out.payDates = schedule.payDates;
            out.dayCount = schedule.dayCount;
            out.dayCountFraction = schedule.dayCountFraction;
            out.numOfPeriod = schedule.numOfPeriod;
        end
        
        function out = numMethodTimeStepsGenerate(rangeAccrual,valueDate,schedule,mcOneTimeStep)
            
            periodCount = schedule.numOfPeriod;
            out.timeStepsPerPeriod =  cell(periodCount,1);
            
            out.startTimeIdx = zeros(periodCount,1);
            out.endM1TimeIdx = zeros(periodCount,1);
            
            out.numOfTimeSteps = zeros(periodCount,1);
            out.totalTimeSteps = [];
            out.totalTimeStepsSize = 0;
            numOfAlivePeriod = 0;
            
            for i=1:periodCount
                timeSteps = [];
                if DateDiff(valueDate,schedule.payDates(i)) >= 0
                    out.timeStepsPerPeriod(i) = timeSteps;
                    continue;
                end
                
                numOfAlivePeriod = numOfAlivePeriod + 1;
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    startDate = valueDate;
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    %out.timeStepsPerPeriod {i,1}.timeSteps = [];
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = 1;
                    out.endM1TimeIdx(i) = length(timeSteps);
                    
                else
                    startDate = schedule.startDates(i);
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = out.endM1TimeIdx(i-1) + 1;
                    out.endM1TimeIdx(i) = out.endM1TimeIdx(i-1) + length(timeSteps);
                    
                end
            end
            
            % endDate of the last period (= Security's maturityDate should
            % be included
            
            endTime = DateDiff(schedule.endDates(periodCount),valueDate);
            out.totalTimeSteps = [out.totalTimeSteps;endTime];
            out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
            out.totalTimeStepsSize = length(out.totalTimeSteps);
            
            out.periodCount = periodCount;
            out.numOfAlivePeriod = numOfAlivePeriod;
        end
        
        function out = generateModelStates(rangeAccrual,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                sqrtDt = sqrt((nextStep-currentStep)/365.0);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                L = chol(stateLocalVariance);
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+2,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+2,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+2,:) = prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = computePriceMCForward(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            
            % numMethodTimeSteps Generate
            mcOneTimeStep = 1;
            outInfo = rangeAccrual.numMethodTimeStepsGenerate(valueDate,schedule,mcOneTimeStep);
            
            numMethodInfo.timeStepsInfo = outInfo;
            
            % mc random number init
            %pathSize = 2^16;
            pathSize = 5000;
            
            %pathSize = 2^10;
            
            rng('default');
            rng(0);
            
            numOfFactors = 2;
            
            Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
            Z=[Z,-Z];
            
            numMethodInfo.pathSize = pathSize;
            numMethodInfo.numOfFactors = numOfFactors;
            numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
            numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
            numMethodInfo.Z = Z;
            
            % numeraire info
            numMatTime = numMethodInfo.totalTimeSteps(end);
            dfNumMat = model.DF(numMatTime/365.0);
            
            %generate model states for the given timeSteps
            modelStates = rangeAccrual.generateModelStates(valueDate,schedule,numMethodInfo,model);
            
            % induct backward and calculate cashflow
            
            
            periodCount = schedule.numOfPeriod;
            
            nonCall.npv = 0;
            
            nonCall.cashflow_e = zeros(periodCount + 1,1);
            nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
            nonCall.rangeProb_e = zeros(periodCount + 1,1);
            
            nonCall.df_e = zeros(periodCount + 1,1);
            nonCall.df_c = zeros(periodCount + 1,1);
            
            % sum of cashflow_npv
            nonCall.payoff = zeros(1,pathSize);
            
            nonCall.payoffStateVectors_cashflow = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVectors_cashflow_npv = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVector_df = zeros(periodCount + 1,pathSize);
            
            payoffStateVectors_cashflow = zeros(1, pathSize);
            payoffStateVectors_cashflow_npv = zeros(1, pathSize);
            payoffStateVectors_df = zeros(1,pathSize);
            
            % we calculate cashflow backward from maturityDate to valueDate
            % rangeAccrual coupons are added at the startDate of the Period
            % At maturity
            
            nominal = rangeAccrual.nominal;
            coupon = rangeAccrual.coupon;
            
            currentTime = numMethodInfo.totalTimeSteps(end);
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            currentTimeIdx = totalTimeStepsSize;
            
            modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
            payoffStateVectors_cashflow = nominal*ones(1,pathSize);
            nonCall.payoffStateVectors_cashflow(end,:) = payoffStateVectors_cashflow;
            
            payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
            nonCall.payoffStateVectors_cashflow_npv(end,:) = payoffStateVectors_cashflow_npv;
            
            nonCall.cashflow_npv_e(end) = mean(payoffStateVectors_cashflow_npv);
            
            nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
            
            payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
            
            nonCall.payoffStateVector_df(end,:) = payoffStateVectors_df;
            nonCall.df_e(end) = mean(nonCall.payoffStateVector_df(end,:));
            nonCall.df_c(end) = model.DF(currentTime/365.0);
%             
%             nonCall.cashflow_npv_df(end,:) =  nonCall.npv/nominal;
%             nonCall.cashflow_npv_df_e(end) = mean(nonCall.cashflow_npv_df(end,:));
            
            lastTimeIdx = currentTimeIdx;
            totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
            
            for i=periodCount:-1:1
                % past schedule neglect
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                endM1TimeIdx = numMethodInfo.timeStepsInfo.endM1TimeIdx(i);
                
                %% FromToInduction start
                
                %modelsStates induct backward(lastTimeIdx to endM1TimeIdx)
                while endM1TimeIdx < lastTimeIdx
                    lastTimeIdx = lastTimeIdx -1;
                    if lastTimeIdx == endM1TimeIdx break;end
                end
                
                currentTimeIdx = lastTimeIdx;
                currentTime = totalTimeSteps(currentTimeIdx);
                % now we are at endM1TimeIdx
                % variables for coupon valuation initialized
                ralower = rangeAccrual.lower;
                raupper = rangeAccrual.upper;
                
                ralowerS = rangeAccrual.lowerS;
                raupperS = rangeAccrual.upperS;
                
                tenorCMS = rangeAccrual.tenor;
                
                tenorLong = rangeAccrual.tenorL;
                tenorShort = rangeAccrual.tenorS;
                
                cD = 0; % cumulated day count fraction
                cP = 0; 
                dI = 0; % dummy idx for loop
                dcfC = 0;
                dcfN = 0;
                dD = 0;
                idxCMS = zeros(1,pathSize);
                
                idxL = zeros(1,pathSize);
                idxS = zeros(1,pathSize);
                
                idxC = zeros(1,pathSize);
                idxN = zeros(1,pathSize);
                
                spreadC = zeros(1,pathSize);
                spreadN = zeros(1,pathSize);
                probSpread = zeros(1,pathSize);
                simpleYN = true;
                numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                
                for j=numOfTimeSteps:-1:1
                    % evaluate dealdescription
                    % last step is omitted
                    currentTime = totalTimeSteps(currentTimeIdx);
                    dcfC = currentTime/365.0;
                    if j~= numOfTimeSteps
                        dD = (dcfN-dcfC);
                    end
                    
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                    idxCMS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                    idxC = idxCMS;
                    
                    idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    spreadC = idxL - idxS;
                     
%                     probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                    probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                    
                    cD = cD + dD;
                    cP = cP + probSpread;
  
                    dcfN = dcfC;
                    
                    idxN = idxC;
                    spreadN = spreadC;
                    
                    %induct backward to the next time step
                    % one time backward induction
                    lastTimeIdx = currentTimeIdx;
                    
                    if startTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        currentTimeIdx = lastTimeIdx;
                    end
                    
                    
                end
                
                %% FromToInduction end
                %% AtDate operation
                % Adding a new cashflow to the payoff
                
                dcf = schedule.dayCountFraction(i);
                payoffStateVectors_cashflow = nominal * coupon * dcf * cP/cD;
                
                % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                % i.e. currentTimeIdx has not changed
                % no need to upodate modelStateT
                % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                % since cashflow's are added at the startDate(i) we need to
                % multiply cashflow with the df(startDate(i), payDate(i))
                payTime = DateDiff(schedule.payDates(i),valueDate);
                df_se = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);
                
                for k=1:pathSize
                    payoffStateVectors_cashflow(k) = payoffStateVectors_cashflow(k) + df_se(k);
                end
                
                nonCall.payoffStateVectors_cashflow(i,:) = payoffStateVectors_cashflow;
                
                nonCall.cashflow_e(i) = mean(payoffStateVectors_cashflow);
                nonCall.rangeProb_e(i) = mean(nonCall.cashflow_e(i)/(nominal * coupon * dcf));
                
                payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
                nonCall.cashflow_npv_e(i) = mean(payoffStateVectors_cashflow_npv);
                
                nonCall.payoffStateVectors_cashflow_npv(i,:) = payoffStateVectors_cashflow_npv;
                nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
                
                payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
                
                nonCall.payoffStateVector_df(i,:) = payoffStateVectors_df;
                nonCall.df_e(i) = mean(nonCall.payoffStateVector_df(i,:));
                nonCall.df_c(i) = model.DF(currentTime/365.0);
                
            end
            
            nonCall.npv = mean(nonCall.payoff);
  
            out.cashflow= nonCall.cashflow_e;
            out.cashflowNpv = nonCall.cashflow_npv_e;
            out.df_e = nonCall.df_e;
            out.df_c = nonCall.df_c;
            out.daycountFraction = schedule.dayCountFraction(i);
            out.rangeProb = nonCall.rangeProb_e;
            out.npv = nonCall.npv;
           
        end
        
        function out = computePrice(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            periodCount = schedule.numOfPeriod;
            rangeProb = zeros(periodCount,1);
            discountFactor = zeros(periodCount,1);
            daycountFraction = zeros(periodCount,1);
            cashflow = zeros(periodCount,1);
            cashflowNpv = zeros(periodCount,1);
            rangeDays = rangeAccrual.rangeDays;
            npv = 0;
            for i=1:periodCount
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end;
                
                observeDate = schedule.startDates(i);
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    observeDate = valueDate;
                    observeDate = AddDate(observeDate,1,'day');
                else
                    observeDate = schedule.startDates(i);
                end
                
                observeDays = DateDiff(schedule.endDates(i),schedule.startDates(i));
                days = DateDiff(schedule.endDates(i), observeDate);
                discountFactor(i) = model.DF( DateDiff(schedule.payDates(i),valueDate)/365.0);
                daycountFraction(i) = schedule.dayCountFraction(i);
                for j=1:days
                    toDate = observeDate;
                    tpDate = schedule.payDates(i);
                    tenor1 = rangeAccrual.tenorL;
                    tenor2 = rangeAccrual.tenorS;
                    tau =0.25;
                    lower = rangeAccrual.lower;
                    upper = rangeAccrual.upper;
                    rangeValue = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper);
%                     rangeValue = 1.0;
                    rangeProb(i) = rangeProb(i) + rangeValue;
                    observeDate = AddDate(observeDate,1,'day');
                end
                cashflow(i) = rangeAccrual.nominal * rangeAccrual.coupon * (rangeProb(i) + rangeDays(i))/observeDays * daycountFraction(i);
                cashflowNpv(i) = cashflow(i) * discountFactor(i);
                npv = npv + cashflowNpv(i);
                % bug fixed observeDays -> days
                rangeProb(i) = rangeProb(i)/days;
                
                if i == periodCount
                    npv = npv + rangeAccrual.nominal * discountFactor(i);
                end
            end
            out.cashflow= cashflow;
            out.cashflowNpv = cashflowNpv;
            out.discountFactor = discountFactor;
            out.daycountFraction = daycountFraction;
            out.rangeProb = rangeProb;
            out.npv = npv;
        end
        
%         function out = discountPayoff(rangeAccrual,eventTime,numMatTime ...
%                 ,model,modelStatesT,cashflow)
%             
%             dfNumMat = model.DF(numMatTime/365.0);
%             % for discountPayoff T=numMatTime
%             % P(0,TnumMat)/P(Te,Tnummat)*Payoff
%             %
%             discountT = model.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
%             stateSize = size(modelStatesT,2);
%             payoff = zeros(1,stateSize);
%             for i=1:stateSize
%                 payoff(i)= dfNumMat/discountT(i)*cashflow(i);
%             end    
%             out = payoff;
%         end
        
        
        
    end
    
end



In [ ]:
classdef CDAndCDKTBSpreadDualRA < Security
    %RangeAccrual is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        nominal
        couponInfo
        callableInfo
        manualScheduleYN
        
    end
    
    methods
        % constructor
        function rangeAccrual = CDAndCDKTBSpreadDualRA(params)
                security = Security(params);
                rangeAccrual.payCcy = security.payCcy;
                rangeAccrual.startDate = security.startDate;
                rangeAccrual.maturity = security.maturity;
                rangeAccrual.endDate = security.endDate;
                
                rangeAccrual.couponInfo = params.params.couponInfo;
                rangeAccrual.callableInfo = params.params.callableInfo;
                rangeAccrual.manualScheduleYN = params.params.manualScheduleYN;
                
                if rangeAccrual.manualScheduleYN
                    rangeAccrual.couponInfo.couponSchedule = rangeAccrual.ManualScheduleGenerate(rangeAccrual.couponInfo.couponScheduleInt);
                    rangeAccrual.callableInfo.callableSchedule = rangeAccrual.ManualScheduleGenerateCall(rangeAccrual.callableInfo.callScheduleInt);
                    
                
                else
                    maturityDate = params.startDate.AddDate(params.maturity,'year');
                    rangeAccrual.schedule = rangeAccrual.ScheduleGenerate(rangeAccrual.startDate,maturityDate,rangeAccrual.couponFreq);

                end
                 
        end
        
        function out = ManualScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.scheduleSize = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.startDates(i) = H_Date(scheduleInt(i,2));
                out.endDates(i) = H_Date(scheduleInt(i,3));
                out.payDates(i) = H_Date(scheduleInt(i,4));
                out.dayCount(i) = out.endDates(i).DateDiff(out.startDates(i));
                out.dayCountFraction(i) = out.dayCount(i)/ 365.0;
            end
            
            
        end
        
        function out = ManualScheduleGenerateCall(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.scheduleSize = scheduleSize;
            for i=1:scheduleSize
                out.startDates(i) = H_Date(scheduleInt(i,1));
                out.payDates(i) = H_Date(scheduleInt(i,2));
            end
            
            
        end
        
        function out = ScheduleGenerate(rangeAccrual,initialDate,maturityDate,freq)
            schedule = ScheduleGenerate(initialDate,maturityDate,freq);
            out.startDates= schedule.startDates;
            out.resetDates = schedule.resetDates;
            out.endDates = schedule.endDates;
            out.payDates = schedule.payDates;
            out.dayCount = schedule.dayCount;
            out.dayCountFraction = schedule.dayCountFraction;
            out.numOfPeriod = schedule.numOfPeriod;
        end
        
        function out = numMethodTimeStepsGenerate(rangeAccrual,valueDate,schedule,mcOneTimeStep)
            
            periodCount = schedule.numOfPeriod;
            out.timeStepsPerPeriod =  cell(periodCount,1);
            
            out.startTimeIdx = zeros(periodCount,1);
            out.endM1TimeIdx = zeros(periodCount,1);
            
            out.numOfTimeSteps = zeros(periodCount,1);
            out.totalTimeSteps = [];
            out.totalTimeStepsSize = 0;
            numOfAlivePeriod = 0;
            
            for i=1:periodCount
                timeSteps = [];
                if DateDiff(valueDate,schedule.payDates(i)) >= 0
                    out.timeStepsPerPeriod(i) = timeSteps;
                    continue;
                end
                
                numOfAlivePeriod = numOfAlivePeriod + 1;
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    startDate = valueDate;
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    %out.timeStepsPerPeriod {i,1}.timeSteps = [];
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = 1;
                    out.endM1TimeIdx(i) = length(timeSteps);
                    
                else
                    startDate = schedule.startDates(i);
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = out.endM1TimeIdx(i-1) + 1;
                    out.endM1TimeIdx(i) = out.endM1TimeIdx(i-1) + length(timeSteps);
                    
                end
            end
            
            % endDate of the last period (= Security's maturityDate should
            % be included
            
            endTime = DateDiff(schedule.endDates(periodCount),valueDate);
            out.totalTimeSteps = [out.totalTimeSteps;endTime];
            out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
            out.totalTimeStepsSize = length(out.totalTimeSteps);
            
            out.periodCount = periodCount;
            out.numOfAlivePeriod = numOfAlivePeriod;
        end
        
        function out = generateModelStates(rangeAccrual,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                sqrtDt = sqrt((nextStep-currentStep)/365.0);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                L = chol(stateLocalVariance);
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+2,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+2,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+2,:) = prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = computePriceMCForward(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            
            % numMethodTimeSteps Generate
            mcOneTimeStep = 1;
            outInfo = rangeAccrual.numMethodTimeStepsGenerate(valueDate,schedule,mcOneTimeStep);
            
            numMethodInfo.timeStepsInfo = outInfo;
            
            % mc random number init
            %pathSize = 2^16;
            pathSize = 5000;
            
            %pathSize = 2^10;
            
            rng('default');
            rng(0);
            
            numOfFactors = 2;
            
            Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
            Z=[Z,-Z];
            
            numMethodInfo.pathSize = pathSize;
            numMethodInfo.numOfFactors = numOfFactors;
            numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
            numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
            numMethodInfo.Z = Z;
            
            % numeraire info
            numMatTime = numMethodInfo.totalTimeSteps(end);
            dfNumMat = model.DF(numMatTime/365.0);
            
            %generate model states for the given timeSteps
            modelStates = rangeAccrual.generateModelStates(valueDate,schedule,numMethodInfo,model);
            
            % induct backward and calculate cashflow
            
            
            periodCount = schedule.numOfPeriod;
            
            nonCall.npv = 0;
            
            nonCall.cashflow_e = zeros(periodCount + 1,1);
            nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
            nonCall.rangeProb_e = zeros(periodCount + 1,1);
            
            nonCall.df_e = zeros(periodCount + 1,1);
            nonCall.df_c = zeros(periodCount + 1,1);
            
            % sum of cashflow_npv
            nonCall.payoff = zeros(1,pathSize);
            
            nonCall.payoffStateVectors_cashflow = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVectors_cashflow_npv = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVector_df = zeros(periodCount + 1,pathSize);
            
            payoffStateVectors_cashflow = zeros(1, pathSize);
            payoffStateVectors_cashflow_npv = zeros(1, pathSize);
            payoffStateVectors_df = zeros(1,pathSize);
            
            % we calculate cashflow backward from maturityDate to valueDate
            % rangeAccrual coupons are added at the startDate of the Period
            % At maturity
            
            nominal = rangeAccrual.nominal;
            coupon = rangeAccrual.coupon;
            
            currentTime = numMethodInfo.totalTimeSteps(end);
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            currentTimeIdx = totalTimeStepsSize;
            
            modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
            payoffStateVectors_cashflow = nominal*ones(1,pathSize);
            nonCall.payoffStateVectors_cashflow(end,:) = payoffStateVectors_cashflow;
            
            payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
            nonCall.payoffStateVectors_cashflow_npv(end,:) = payoffStateVectors_cashflow_npv;
            
            nonCall.cashflow_npv_e(end) = mean(payoffStateVectors_cashflow_npv);
            
            nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
            
            payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
            
            nonCall.payoffStateVector_df(end,:) = payoffStateVectors_df;
            nonCall.df_e(end) = mean(nonCall.payoffStateVector_df(end,:));
            nonCall.df_c(end) = model.DF(currentTime/365.0);
%             
%             nonCall.cashflow_npv_df(end,:) =  nonCall.npv/nominal;
%             nonCall.cashflow_npv_df_e(end) = mean(nonCall.cashflow_npv_df(end,:));
            
            lastTimeIdx = currentTimeIdx;
            totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
            
            for i=periodCount:-1:1
                % past schedule neglect
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                endM1TimeIdx = numMethodInfo.timeStepsInfo.endM1TimeIdx(i);
                
                %% FromToInduction start
                
                %modelsStates induct backward(lastTimeIdx to endM1TimeIdx)
                while endM1TimeIdx < lastTimeIdx
                    lastTimeIdx = lastTimeIdx -1;
                    if lastTimeIdx == endM1TimeIdx break;end
                end
                
                currentTimeIdx = lastTimeIdx;
                currentTime = totalTimeSteps(currentTimeIdx);
                % now we are at endM1TimeIdx
                % variables for coupon valuation initialized
                ralower = rangeAccrual.lower;
                raupper = rangeAccrual.upper;
                
                ralowerS = rangeAccrual.lowerS;
                raupperS = rangeAccrual.upperS;
                
                tenorCMS = rangeAccrual.tenor;
                
                tenorLong = rangeAccrual.tenorL;
                tenorShort = rangeAccrual.tenorS;
                
                cD = 0; % cumulated day count fraction
                cP = 0; 
                dI = 0; % dummy idx for loop
                dcfC = 0;
                dcfN = 0;
                dD = 0;
                idxCMS = zeros(1,pathSize);
                
                idxL = zeros(1,pathSize);
                idxS = zeros(1,pathSize);
                
                idxC = zeros(1,pathSize);
                idxN = zeros(1,pathSize);
                
                spreadC = zeros(1,pathSize);
                spreadN = zeros(1,pathSize);
                probSpread = zeros(1,pathSize);
                simpleYN = true;
                numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                
                for j=numOfTimeSteps:-1:1
                    % evaluate dealdescription
                    % last step is omitted
                    currentTime = totalTimeSteps(currentTimeIdx);
                    dcfC = currentTime/365.0;
                    if j~= numOfTimeSteps
                        dD = (dcfN-dcfC);
                    end
                    
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                    idxCMS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                    idxC = idxCMS;
                    
                    idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    spreadC = idxL - idxS;
                     
%                     probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                    probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                    
                    cD = cD + dD;
                    cP = cP + probSpread;
  
                    dcfN = dcfC;
                    
                    idxN = idxC;
                    spreadN = spreadC;
                    
                    %induct backward to the next time step
                    % one time backward induction
                    lastTimeIdx = currentTimeIdx;
                    
                    if startTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        currentTimeIdx = lastTimeIdx;
                    end
                    
                    
                end
                
                %% FromToInduction end
                %% AtDate operation
                % Adding a new cashflow to the payoff
                
                dcf = schedule.dayCountFraction(i);
                payoffStateVectors_cashflow = nominal * coupon * dcf * cP/cD;
                
                % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                % i.e. currentTimeIdx has not changed
                % no need to upodate modelStateT
                % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                % since cashflow's are added at the startDate(i) we need to
                % multiply cashflow with the df(startDate(i), payDate(i))
                payTime = DateDiff(schedule.payDates(i),valueDate);
                df_se = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);
                
                for k=1:pathSize
                    payoffStateVectors_cashflow(k) = payoffStateVectors_cashflow(k) + df_se(k);
                end
                
                nonCall.payoffStateVectors_cashflow(i,:) = payoffStateVectors_cashflow;
                
                nonCall.cashflow_e(i) = mean(payoffStateVectors_cashflow);
                nonCall.rangeProb_e(i) = mean(nonCall.cashflow_e(i)/(nominal * coupon * dcf));
                
                payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
                nonCall.cashflow_npv_e(i) = mean(payoffStateVectors_cashflow_npv);
                
                nonCall.payoffStateVectors_cashflow_npv(i,:) = payoffStateVectors_cashflow_npv;
                nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
                
                payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
                
                nonCall.payoffStateVector_df(i,:) = payoffStateVectors_df;
                nonCall.df_e(i) = mean(nonCall.payoffStateVector_df(i,:));
                nonCall.df_c(i) = model.DF(currentTime/365.0);
                
            end
            
            nonCall.npv = mean(nonCall.payoff);
  
            out.cashflow= nonCall.cashflow_e;
            out.cashflowNpv = nonCall.cashflow_npv_e;
            out.df_e = nonCall.df_e;
            out.df_c = nonCall.df_c;
            out.daycountFraction = schedule.dayCountFraction(i);
            out.rangeProb = nonCall.rangeProb_e;
            out.npv = nonCall.npv;
           
        end
        
        function out = computePrice(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            periodCount = schedule.numOfPeriod;
            rangeProb = zeros(periodCount,1);
            discountFactor = zeros(periodCount,1);
            daycountFraction = zeros(periodCount,1);
            cashflow = zeros(periodCount,1);
            cashflowNpv = zeros(periodCount,1);
            rangeDays = rangeAccrual.rangeDays;
            npv = 0;
            for i=1:periodCount
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end;
                
                observeDate = schedule.startDates(i);
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    observeDate = valueDate;
                    observeDate = AddDate(observeDate,1,'day');
                else
                    observeDate = schedule.startDates(i);
                end
                
                observeDays = DateDiff(schedule.endDates(i),schedule.startDates(i));
                days = DateDiff(schedule.endDates(i), observeDate);
                discountFactor(i) = model.DF( DateDiff(schedule.payDates(i),valueDate)/365.0);
                daycountFraction(i) = schedule.dayCountFraction(i);
                for j=1:days
                    toDate = observeDate;
                    tpDate = schedule.payDates(i);
                    tenor1 = rangeAccrual.tenorL;
                    tenor2 = rangeAccrual.tenorS;
                    tau =0.25;
                    lower = rangeAccrual.lower;
                    upper = rangeAccrual.upper;
                    rangeValue = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper);
%                     rangeValue = 1.0;
                    rangeProb(i) = rangeProb(i) + rangeValue;
                    observeDate = AddDate(observeDate,1,'day');
                end
                cashflow(i) = rangeAccrual.nominal * rangeAccrual.coupon * (rangeProb(i) + rangeDays(i))/observeDays * daycountFraction(i);
                cashflowNpv(i) = cashflow(i) * discountFactor(i);
                npv = npv + cashflowNpv(i);
                % bug fixed observeDays -> days
                rangeProb(i) = rangeProb(i)/days;
                
                if i == periodCount
                    npv = npv + rangeAccrual.nominal * discountFactor(i);
                end
            end
            out.cashflow= cashflow;
            out.cashflowNpv = cashflowNpv;
            out.discountFactor = discountFactor;
            out.daycountFraction = daycountFraction;
            out.rangeProb = rangeProb;
            out.npv = npv;
        end
        
%         function out = discountPayoff(rangeAccrual,eventTime,numMatTime ...
%                 ,model,modelStatesT,cashflow)
%             
%             dfNumMat = model.DF(numMatTime/365.0);
%             % for discountPayoff T=numMatTime
%             % P(0,TnumMat)/P(Te,Tnummat)*Payoff
%             %
%             discountT = model.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
%             stateSize = size(modelStatesT,2);
%             payoff = zeros(1,stateSize);
%             for i=1:stateSize
%                 payoff(i)= dfNumMat/discountT(i)*cashflow(i);
%             end    
%             out = payoff;
%         end
        
        
        
    end
    
end



In [ ]:
function out = CapletHWType2(zeroRate,fwdRate,strike,valueDate,resetDate,endDate,payDate,sigma,kappa,fixingAccraulFactor)

    resetTime = DateDiff(resetDate,valueDate)/365.0;
    endTime = DateDiff(endDate,valueDate)/365.0;
    payTime = DateDiff(payDate,valueDate)/365.0;

    df = exp(-1.0*zeroRate*payTime);
    
    if endTime < 0 
        out = 0;
        return;
        
    elseif endTime == 0 % option expiry is endTime
        out = max(fwdRate-strike,0);
        return;
    end
    
    if resetTime < 0  % option is expired
        
        deltaTime = endTime-resetTime;
        aa = 4.0 * exp(-1.0*kappa*endTime) - 1.0 * exp(-2.0*kappa*endTime) + 2.0 *kappa*endTime - 3.0;
        kappaTo3 = kappa*kappa*kappa;
        variance = sigma*sigma/2.0/kappaTo3*aa;
        vol = sqrt(variance);
        d1 = log((1.0 + deltaTime*fwdRate)/(1.0+deltaTime*strike))/vol + 0.5*vol;
        d2 = d1 - vol;
        out = (1.0 + deltaTime*fwdRate)*H_ncdf(d1) - (1.0 + deltaTime*strike)*H_ncdf(d2);
        out = out * df;
    else
    
        deltaTime = endTime-resetTime;
        aa = (exp(-1.0*kappa*deltaTime) + kappa*deltaTime - 1.0);
        bb = (1.0-exp(-1.0*kappa*deltaTime));
        cc = 2.0 * aa -1.0 * bb *bb *exp(-2.0*kappa*resetTime);
        kappaTo3 = kappa*kappa*kappa;
        variance = sigma*sigma/2.0/kappaTo3*cc;
        vol = sqrt(variance);
        d1 = log((1.0 + deltaTime*fwdRate)/(1.0+deltaTime*strike))/vol + 0.5*vol;
        d2 = d1 - vol;
        out = (1.0 + deltaTime*fwdRate)*H_ncdf(d1) - (1.0 + deltaTime*strike)*H_ncdf(d2);
        out = out * df;
        
        
    end
    
end



In [ ]:
function out = CapletHWType1(zeroRate,fwdRate,strike,valueDate,resetDate,endDate,payDate,sigma,kappa,fixingAccraulFactor)

    resetTime = DateDiff(resetDate,valueDate)/365.0;
    endTime = DateDiff(endDate,valueDate)/365.0;
    payTime = DateDiff(payDate,valueDate)/365.0;

    df = exp(-1.0*zeroRate*payTime);
    
    if resetTime < 0  % option is expired
        out = 0;
        return;
        
    elseif resetTime == 0 % at option expiry 
        out = max(fwdRate-strike,0);
        return;
    else
    
        deltaTime = endTime-resetTime;
        aa = (exp(-1.0*kappa*deltaTime) - 1.0);
        bb = (1.0-exp(-2.0*kappa*resetTime));
        kappaTo3 = kappa*kappa*kappa;
        variance = sigma*sigma/2.0/kappaTo3*aa*aa*bb;
        vol = sqrt(variance);
        d1 = log((1.0 + deltaTime*fwdRate)/(1.0+deltaTime*strike))/vol + 0.5*vol;
        d2 = d1 - vol;
        out = (1.0 + deltaTime*fwdRate)*H_ncdf(d1) - (1.0 + deltaTime*strike)*H_ncdf(d2);
        out = out * df;
        
        
    end
    
end



In [ ]:
function [P] = Call_RoughHeston_MF(S0,K,T,r,x,n,r_n)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% This f u n c t i o n p r i c e s a Ca l l o p t i on us ing Lewis ’ approach and
% Adams method f o r t h e c h a r a c t e r i s t i c func t i on , c o n s i d e r i n g
% t h e mul t i􀀀f a c t o r Heston model f o r t h e v o l a t i l i t y
%
%
% INPUT:
% S0 : i n i t i a l s p o t p r i c e
% K : s t r i k e p r i c e
% T : t ime t o e x p i r y ( in y e a r s )
% r : r i s k f r e e r a t e
% x : nu , lambda , rho , t h e t a ,V0 , a l ph a
% Rough Heston parame t e r s
% n : number o f f a c t o r s
% r_n : mu l t i f a c t o r c o e f f i c i e n t
%
%
% OUTPUT:
% P : European Ca l l p r i c e
%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%f u n c t i o n handl e f o r t h e Four i e r t rans form o f t h e
%c h a r a c t e r i s t i c f u n c t i o n
KTilda = log(S0/K) + r*T;

% integrand=@(u)(real(tr_fourier_MF(x,T,u,n,r_n).*exp(1i*log(K)*u)));
integrand=@(u)(real(tr_fourier_MF(x,T,u,n,r_n).*exp(1i*KTilda*u)));

%e x p l i c i t i n t e g r a t i o n
phiL = integral(integrand,0,100) ;

%Lewis ’ Pr i c ing formula
P = S0 -((sqrt(S0*K)*exp(-r*T/2)/pi)*phiL) ;

end

In [ ]:
function [P] = Call_RoughHeston ( S0 ,K,T, r , x )

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% This f u n c t i o n p r i c e s a Ca l l o p t i on us ing Lewis ’ approach and
% Adams ’ method f o r t h e c h a r a c t e r i s t i c func t i on , c o n s i d e r i n g
% t h e Rough Heston model f o r t h e v o l a t i l i t y
%
%
% INPUT:
% S0 : i n i t i a l s p o t p r i c e
% K : s t r i k e p r i c e
% T : t ime t o e x p i r y ( in y e a r s )
% r : r i s k f r e e r a t e
% x : Rough Heston parame t e r s
%
%
% OUTPUT:
% P : European Ca l l p r i c e
%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

%f u n c t i o n handl e f o r t h e Four i e r t rans form o f t h e
%c h a r a c t e r i s t i c f u n c t i o n
KTilda = log(S0/K) + r*T;

% integrand=@(u) ( real( tr_fourier(x ,T, u).*exp(1i*log(K)*u) ) ) ;
integrand=@(u) ( real( tr_fourier(x ,T, u).*exp(1i*KTilda*u) ) ) ;

%e x p l i c i t i n t e g r a t i o n

phiL = integral( integrand , 0 , 100) ;

%Lewis ’ Pr i c ing formula
P = S0 - ( ( sqrt(S0*K)*exp(-r*T/2)/pi )*phiL ) ;

end

In [ ]:
function P = Call_Price_Heston (S,K,T,r,kappa,theta,sigma,rho,V,alpha,L)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% This f u n c t i o n p r i c e s a Ca l l o p t i on wi t h Matur i ty T and
% S t r i k e K us ing t h e c h a r a c t e r i s t i c f u n c t i o n o f t h e p r i c e and
% Lewis ’ approach
%
% No FFT used
%
%
% INPUT:
% S : i n i t i a l s p o t p r i c e
% r : r i s k f r e e r a t e
% kappa , t h e t a , sigma , rho : Heston model parame t e r s
% V : i n i t i a l v o l in Heston model
% al p h a : damping f a c t o r ( a l ph a >0)
% L : t r u n c a t i o n bound f o r t h e i n t e g r a l
%
%
% OUTPUT:
% P : European Ca l l p r i c e
%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%Heston c h a r a c t e r i s t i c f u n c t i o n
% 
% b=@(nu) ( kappa-1i*rho*sigma.*nu) ;
% gamma=@(nu) ( sqrt( sigma^2*(nu.^2+1i.*nu)+b(nu).^2));
% a=@(nu) (b(nu)./gamma(nu)).*sinh(T*0.5.*gamma(nu));
% c=@(nu) (gamma(nu).*coth(0.5*T.*gamma(nu))+b(nu));
% d=@(nu) ( kappa*theta*T.*b(nu)/sigma^2);
% 
% f=@(nu) (1i*(log(S)+r*T).*nu+d(nu));
% g=@(nu) (cosh(T*0.5.*gamma(nu))+a(nu)).^(2*kappa*theta/sigma^2) ;
% h=@(nu) (-(nu.^2+1i.*nu)*V./c(nu));

% phi=@(nu)(exp(f(nu)).*exp(h(nu))./g(nu));
Trap = 1;
lambda = 0.0;
tau = T;
q= 0.0;
v0 = V;
trap = 1;

load('laguerre192.mat','laguerre192');
quadrature_nodesAndWeights = laguerre192;
x = quadrature_nodesAndWeights(:,1);
w = quadrature_nodesAndWeights(:,2);
            
% [x w] = GenerateGaussLaguerre(32);

P  = LewisPrice311(S,K,r,q,T,theta,kappa,sigma,rho,v0,trap,x,w);
% 
% 
% phi=@(u)(LewisIntegrand311(u,kappa,theta,lambda,rho,sigma,tau,S,K,r,q,v0,Trap));
% 
% KTilda = log(S/K) + r*T;
% %f u n c t i o n handl e f o r t h e Four i e r t rans form o f t h e
% %c h a r a c t e r i s t i c f u n c t i o n
% % integrand=@(u)(real(phi(u-1i/2).*exp(1i*log(K)*u))./(u.^2+.25));
% % integrand=@(u)(real(phi(u-1i/2).*exp(1i*KTilda*u))./(u.^2+.25));
% integrand=@(u)(phi(u));
% 
% %e x p l i c i t i n t e g r a t i o n
% phi1 = integral(integrand,0,L) ;
% 
% 
% % Lewis ’ Pr i c ing formula
% P = S- ((sqrt(S*K)*exp(-r*T/2)/pi)*phi1);

end

In [ ]:
%calibration_lgm1f_CalibrateToATMSwaptionTenorDiagSwaption.m

clc
clear
% valueDate = '2018-01-09';
valueDate = '2017-02-17';

%% USD model building start
%% import and construct a zeroCurve
rawData = [0.002739726	0.999960071
0.083333333	0.998705285
0.166666667	0.997278569
0.25	0.99570906
0.5	0.990659472
0.75	0.985691562
1	0.980706602
2	0.958190889
3	0.935244778
4	0.91270743
5	0.890529514
7	0.846476648
10	0.781398058
12	0.739443263
15	0.680504997
20	0.593081133
25	0.519315395
30	0.456759825
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'USD','USDZero'});
zeroCurveMktData = MktData(mktData);
zeroCurveMktData.params('convention') = {};
zeroCurve =  ZeroCurve(zeroCurveMktData);
USDAllMktData =containers.Map('zeroCurve',zeroCurve);

%% import and construct a discountCurve
rawData = [0.002739726	0.999960071
0.083333333	0.998705285
0.166666667	0.997278569
0.25	0.99570906
0.5	0.990659472
0.75	0.985691562
1	0.980706602
2	0.958190889
3	0.935244778
4	0.91270743
5	0.890529514
7	0.846476648
10	0.781398058
12	0.739443263
15	0.680504997
20	0.593081133
25	0.519315395
30	0.456759825
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'USD','discountZero'});
discountCurveMktData = MktData(mktData);
discountCurveMktData.params('convention') = {};
discountCurve =  ZeroCurve(discountCurveMktData);
USDAllMktData('discountCurve') = discountCurve;

%% import and construct a swaptionVolSurface
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.18785	0.21535	0.228	0.23405	0.23705	0.23735	0.2341	0.23	0.2259
2	0.22675	0.24245	0.25075	0.25335	0.25235	0.2504	0.2464	0.23945	0.2325
3	0.25885	0.26305	0.2647	0.263	0.26135	0.2569	0.2502	0.243325	0.23645
4	0.2736	0.27245	0.2692	0.266	0.263	0.25715	0.2503	0.243125	0.23595
5	0.27605	0.272	0.26905	0.26565	0.2627	0.2561	0.24885	0.24125	0.23365
7	0.2656	0.26325	0.25995	0.25655	0.25355	0.24835	0.24125	0.233575	0.2259
10	0.24575	0.2439	0.24245	0.23995	0.2385	0.23425	0.22875	0.2217	0.21465
15	0.2192	0.21675	0.2156	0.21345	0.21175	0.20865	0.20585	0.2007	0.19555
    ];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawDataSwaption,valueDate,'USD','USDSwaptionVol'});
swaptionVolMktData = MktData(mktData);
USDAllMktData('swaptionVol') = swaptionVolMktData;

%% create  a BlackModel to calculate market Swaption
modelParams= containers.Map({'modelName'},{'Black'});
USDBlack = IRBlack(IRModel(Model(USDAllMktData,modelParams))); 

expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
swaptionPriceSurf.expiry= expiry;
swaptionPriceSurf.tenor = tenor;

tempdHTenor = zeros(size(expiry,1),size(tenor,1));
for i=1:size(expiry,1)
    for j=1:size(tenor,1)
%         swaptionPriceSurf.surface(i,j)= USDBlack.BlackATMSwaption(expiry(i),tenor(j));
%         swaptionVolSurf(i,j) = USDBlack.swaptionVol(expiry(i),tenor(j));
        tempdHTenor(i,j) = expiry(i) + tenor(j);
    end
end
%% create a LGM1FModel with initial modelParams

dH1.tenor = [1;2;3;4;5;7;10;15];

dH1.quote = ones(1,length(dH1.tenor))*1.0;
dH1Prev = dH1;
Alpha1.tenor =  [1;2;3;4;5;7;10;15];
Alpha1.quote = ones(1,length(dH1.tenor))*0.01;

joint_swap_tenor = 10;

smoothdH = 1.0;
smoothAlpha = 0.00;

targetSize = 15;

freq = 4;
matu = 15 + 1.0/365.0;

tic
modelParams= containers.Map({'dH1','Alpha1','matu','joint_swap_tenor','modelName','dH1Prev','smoothdH','smoothAlpha','targetSize','freq'}, ...
                        {dH1,Alpha1,matu,joint_swap_tenor, 'LGM1F',dH1Prev,smoothdH,smoothAlpha,targetSize,freq});
USDLGM1F = LGM1F(IRModel(Model(USDAllMktData,modelParams)));

USDLGM = LGM1F(IRModel(Model(USDAllMktData,modelParams)));
out = USDLGM.CalibrateToATMSwaptionTenorDiagSwaption(USDBlack,'global');
% out = USDLGM1F.CalibrateToATMDiagSwaption(KRWBlack,'global');
aaa =1.0;

dH1Calib = USDLGM.dH1;
Alpha1Calib = USDLGM.Alpha1;

modelParamsCalib= containers.Map({'dH1','Alpha1','matu','joint_swap_tenor','modelName','dH1Prev','smoothdH','smoothAlpha','targetSize','freq'}, ...
                    {dH1Calib,Alpha1Calib,matu,joint_swap_tenor,'LGM1F',dH1Prev,smoothdH,smoothAlpha,targetSize,freq});

USDLGMRisky = LGM1FRisky(IRModelRisky(Model(USDAllMktData,modelParamsCalib)));

%% USD model building end



In [ ]:
classdef Calendar < H_RootObject

    

    properties(SetAccess = public)
        name
        holidaysList

    end

    

    methods

        function calendar = Calendar()
            calendar.name = '';
            calendar.holidaysList = {};

        end

        

        function AddHoliday(calendar,h_Date)

            if strcmp(class(h_Date), 'H_Date')
                if calendar.IsBusinessDay(h_Date)    
                    calendar.holidaysList{end+1}=h_Date;
                end
            elseif strcmp(class(h_Date),'char') || strcmp(class(h_Date),'string')
                h_Date1 = H_Date(h_Date);
                if calendar.IsBusinessDay(h_Date1)
                    calendar.holidaysList{end+1}=h_Date1;
                end
            else

                error('wrong date type')

            end

        end

        

        function out = IsHoliday(calendar,h_Date)

            if strcmp(class(h_Date),'char') || strcmp(class(h_Date),'string')

                h_Date = H_Date(h_Date);

            end

            

            if IsWeekend(h_Date)

                out = true;

                return;

            else

                index = cellfun(@(x) x==h_Date, calendar.holidaysList, ...
                        'UniformOutput', 1);

                idx = find(index);

                if ~isempty(idx) 

                    out = true;

                    return;

                end

            end

            out = false;

        end

        

        function out = IsBusinessDay(calendar,h_Date)

            out = ~IsHoliday(calendar,h_Date);

        end

        

        function out = Adjust(calendar,d,bdc)

            if strcmp(bdc,'Unadjusted')

                out = d;

                return;

            end

            d1 = d;

            if strcmp(bdc,'Following') || ...
               strcmp(bdc,'ModifiedFollowing') || ...
               strcmp(bdc,'HalfMonthModifiedFollowing')

               while IsHoliday(calendar,d1)

                   d1 = d1.AddDate(1,'day');

               end

               

               if strcmp(bdc,'ModifiedFollowing') || ...
                    strcmp(bdc,'HalfMonthModifiedFollowing')

                   if d1.month ~= d.month 

                       out = calendar.Adjust(d, 'Preceding');

                       return;

                   end

                   

                   if strcmp(bdc,'HalfMonthModifiedFollowing')

                       if d.day <= 15 && d1.dayOfMonth > 15

                           out = calendar.Adjust(d, 'Preceding');

                           return;

                       end

                   end

                   

               end

            elseif strcmp(bdc,'Preceding') || ...
                   strcmp(bdc,'ModifiedPreceding')

               while IsHoliday(calendar,d1)

                   d1 = d1.AddDate(-1,'day');

               end

               

               if strcmp(bdc,'ModifiedPreceding') && ...
                       d1.month ~= d.month

                   out = calendar.Adjust(d,'Following');

                   return;

               end

            else

                error('unknown business day convention');

            end

            out = d1;

            

        end

        function out = AdvanceTenor(calendar,d,bdc,tenor,endOfMonth)
           if strcmp(tenor(end),'D')
                out = Advance(calendar,d,bdc,str2double(tenor(1:end-1)),'day',endOfMonth);
           elseif strcmp(tenor(end),'M')
                out = Advance(calendar,d,bdc,str2double(tenor(1:end-1)),'month',endOfMonth);
           elseif strcmp(tenor(end),'Y')
               out = Advance(calendar,d,bdc,str2double(tenor(1:end-1)),'year',endOfMonth);
           else
               error('unimplemented')
           end 
        end

        function out = Advance(calendar,d,bdc,n,unit,endOfMonth)

            if n == 0 

                out =  calendar.Adjust(d,bdc);

            elseif strcmp(unit,'day')

                d1 = d;

                if n > 0

                    while n > 0

                        d1 = d1.AddDate(1,'day');

                        while IsHoliday(calendar,d1)

                            d1 = d1.AddDate(1,'day');

                        end

                        n = n-1;

                    end

                else 

                    while n < 0

                        d1 = d1.AddDate(-1,'day');

                        while IsHoliday(calendar,d1)

                            d1 = d1.AddDate(-1,'day');

                        end

                        n = n +1;

                    end

                end

                

                out = d1;

                return;

            else

                d1 = d.AddDate(n,unit);

                % we are sure the unit is Months or Years

                if endOfMonth && IsEndOfMonth(d)

                    out =calendar.Adjust(EndOfMonth(d1),'Preceding');

                    return;

                end

                

                out =  Adjust(calendar,d1, bdc);

                return;

            end

        end

    end

    

end





In [ ]:
function rlag=calculateReturns(prices, lag)
% rlag=calculateReturns(prices, lag) returns the lagged returns based on
% the price series

% rlag=log(prices)-backshift(lag, log(prices));
prevPrices=backshift(lag, prices);
rlag=(prices-prevPrices)./prevPrices;




In [ ]:
function output = BootStrapZeroFromYC(ycData,date,curveName,globalDict)
    
    nowDate = date;
    %% get calendars
    if  isa(nowDate,'H_Date')
        nowDateH = nowDate;
    else
        nowDateH = H_Date(nowDate);
    end


%     fromDateStr = int2str(FngDate(nowDateH));
%     DataNm = curveName;
%     curveType = 'market';
%     yieldCurveMarketQueryStr1 = 'http://45.12.71.111:50100/data/market/yieldcurves/';
% 
%     queryStr = strcat(yieldCurveMarketQueryStr1,DataNm,'/',fromDateStr,'?type=',curveType);
%     dataJson = webread(queryStr);
% 
%     dataType = cell(length(dataJson.Yields),1);
%     dataTenor = cell(length(dataJson.Yields),1);
%     dataRate = cell(length(dataJson.Yields),1);
%     
%     for i=1:length(dataJson.Yields)
%         sizeStr = int2str(cast(dataJson.Yields(i).Tenor.Value,'int32'));
%         periodTypeStr = dataJson.Yields(i).Tenor.TenorCd.Cd;
%         dataTenor{i} = strcat(sizeStr,periodTypeStr);
%         dataType{i} = dataJson.Yields(i).YcInsTpNm;
%         dataRate{i} = dataJson.Yields(i).Rate;
%     end
%     
%     ycData = {};
%     ycData.dataType = dataType;
%     ycData.dataTenor = dataTenor;
%     ycData.dataRate = dataRate;
%     
%     output.ycData = ycData;
    
    if 1
        
        if strcmp(curveName,'KRW_KTB')
            ccy = 'KTB';
        else
            ccy = curveName(1:3);
        end
    
        yieldCurveSettings = globalDict('yieldCurveSettings');
        if isKey(yieldCurveSettings,curveName)
            yieldCurveParams = yieldCurveSettings(curveName);
        else
            yieldCurveParams = yieldCurveSettings('KRW_KTB');
        end
        
        zcCurveOut = YieldCurveBootStrap(ccy,nowDateH,ycData,yieldCurveParams);
        
        output.zcCurve = zcCurveOut;
    
    else
        output.zcCurve = ZeroCurve(MktData(containers.Map('asOfDate',nowDateH)));
        
    end
    
end





In [ ]:
function y = BlackFwdVega(F,K,vol,T)

% Bisection algorithm
% PutCall = "P"ut or "C"all
% F = Forward price
% K = Strike price
% vol = volatility
% T = Maturity

    d1= (log(F/K)+(vol^2/2.0)*T)/(vol*sqrt(T));
    
    y = sqrt(T)*H_ndf(d1);

    
end



In [ ]:
function y = BlackFwdDelta(F,K,vol,T)

% Bisection algorithm
% PutCall = "P"ut or "C"all
% F = Forward price
% K = Strike price
% vol = volatility
% T = Maturity

    d1= (log(F/K)+(vol^2/2.0)*T)/(vol*sqrt(T));
    
    y = -1.0*H_ncdf(-d1);

    
end



In [ ]:
classdef BlackFormula
    % Jim Gatheral R Code to Matlab
    % Convert each R library as blank class(function container)
    
    methods
        
%         function obj = BlackFormula()
%                 
%         end
        
        function out = BlackCall(obj,S0, K, T, sigma)
            k  = log(K./S0);
            sig= sigma.*sqrt(T);
            d1 = -k./sig+sig./2;
            d2 = d1 - sig;
            out = S0.*H_ncdf(d1) - K.*H_ncdf(d2);
            
        end
        
        function out = BlackPut(obj,S0, K, T, sigma)
            k  = log(K./S0);
            sig= sigma.*sqrt(T);
            d1 = -k./sig+sig/2;
            d2 = d1 - sig;
            out = K.*H_ncdf(-d2) - S0.*H_ncdf(-d1);
        end
        
        function out = ivCall(obj,S0, K, T, C)
            nK = length(K);
%             sigmaL = ones(nK,1)*1e-10;
            sigmaL = 1e-10;
            
            CL  = obj.BlackCall(S0, K, T, sigmaL);
%             sigmaH =ones(nK,1)*10.0;
            sigmaH =10.0;
            
            CH  = obj.BlackCall(S0, K, T, sigmaH);

            while (mean(sigmaH - sigmaL) > 1e-10)
                sigma = (sigmaL + sigmaH)./2;
                CM = obj.BlackCall(S0, K, T, sigma);
                CL = CL + (CM < C).*(CM-CL);
                sigmaL = sigmaL + (CM < C).*(sigma-sigmaL);
                CH = CH + (CM >= C).*(CM-CH);
                sigmaH = sigmaH + (CM >= C).*(sigma-sigmaH);
            end

            out = sigma;
        
        end
        
        function out = ivPut(obj,S0, K, T, P)
            % This function also works with vectors of strikes and option values  
            
            nK = length(K);
            sigmaL = 1e-10;
            PL  = obj.BlackPut(S0, K, T, sigmaL);
            sigmaH =10.0;
            PH  = obj.BlackPut(S0, K, T, sigmaH);

            while (mean(sigmaH - sigmaL) > 1e-10)
                sigma = (sigmaL + sigmaH)./2;
                PM = obj.BlackPut(S0, K, T, sigma);
                PL = PL + (PM < P).*(PM-PL);
                sigmaL = sigmaL + (PM < P).*(sigma-sigmaL);
                PH = PH + (PM >= P).*(PM-PH);
                sigmaH = sigmaH + (PM >= P).*(sigma-sigmaH);
            end

            out = sigma;
        
        end
        
        function out = ivOTM(obj,S0, K, T, V)
%             if (K > S0) out = obj.ivCall(S0,K,T,V); else out = obj.ivPut(S0,K,T,V);end;
            out = (S0 < K).*obj.ivCall(S0,K,T,V) + (S0 >= K).* obj.ivPut(S0,K,T,V);
        end
        
%         function out = ivOTM(obj,S0, K, T, V)
%             out = arrayfun(@(K) obj.ivOTMRaw(S0, K, T, V),K);
%         end
        
        function out = ivS(obj,Sf,T,AK)
            % Function to compute option prices and implied vols
            % given vector of final values of underlying
            
            nK = length(AK);
            N  = length(Sf);
            Sfbar = mean(Sf);
            V =ones(1,nK);
            ivBlack =ones(1,nK);
            for j= 1:nK 
                payoff = (Sf - AK(j)) .* (Sf > AK(j));
                V = mean(payoff);
                if (Sfbar< AK(j)) V_OTM = V; else V_OTM = V + AK(j) - Sfbar;end;
                ivBlack(j) =obj.ivOTM(Sfbar, AK(j), T, V_OTM);
            end
            out =  ivBlack;
        end
    end

end



In [ ]:
function y = BisecNormIV(PutCall,S,K,rf,T,a,b,MktPrice,Tol,MaxIter)

% Bisection algorithm
% PutCall = "P"ut or "C"all
% S = Spot price
% K = Strike price
% rf = Risk free rate
% T = Maturity
% a = Starting point lower value of vol
% b = Starting point upper value of vol
% MktPrice = Market price
% Tol = Tolerance
% MaxIter = maximum number of iterations

% Functions for the Black-Scholes call and put  
NormC = @(s,K,rf,v,T) (exp(-rf*T)*((s*exp(rf*T)-K)*H_ncdf((s*exp(rf*T)-K)/v./sqrt(T)) + v.*sqrt(T)*H_ndf((s*exp(rf*T)-K)/v./sqrt(T))));
NormP = @(s,K,rf,v,T) (exp(-rf*T)*((K-s*exp(rf*T))*H_ncdf((K-s*exp(rf*T))/v./sqrt(T)) + v.*sqrt(T)*H_ndf((s*exp(rf*T)-K)/v./sqrt(T))));
  
if strcmp(PutCall,'C')
	lowCdif  = MktPrice - NormC(S,K,rf,a,T);
	highCdif = MktPrice - NormC(S,K,rf,b,T);
else
	lowCdif  = MktPrice - NormP(S,K,rf,a,T);
	highCdif = MktPrice - NormP(S,K,rf,b,T);
end

if lowCdif*highCdif > 0
	y = -1;
else
	for x=1:MaxIter
		midP = (a+b)/2;
		if strcmp(PutCall,'C')
			midCdif = MktPrice - NormC(S,K,rf,midP,T);
		else
			midCdif = MktPrice - NormP(S,K,rf,midP,T);
		end
		if abs(midCdif)<Tol
			break
		else
			if midCdif>0
				a = midP;
			else
				b = midP;
			end
		end
	end
	y = midP;
end



In [ ]:
function y = BisecFwdMoneynessPADelta(delta,sig,T)

% Bisection algorithm
% delta
% sig = volatility
% T = Maturity
% a = Starting point lower value of fwdmoneyness
% b = Starting point upper value of fwdmoneyness
% Tol = Tolerance
% MaxIter = maximum number of iterations

% Tol = 1e-6;
Tol = 1e-10;
MaxIter =1000;


% Functions for the Black-Scholes call and put  
deltaPAC = @(k,v,T) ( 1.0*k*H_ncdf( 1.0*(-log(k)-0.5*v*v*T)/(v*sqrt(T))));
deltaPAP = @(k,v,T) (-1.0*k*H_ncdf(-1.0*(-log(k)-0.5*v*v*T)/(v*sqrt(T))));

% ATM
if abs(delta - 0.5) < 0.0001
    y = exp(-0.5*sig*sig*T);
    return;
end

if delta > 0 % Call
    a = 1.0;
    b = 2.0;
	lowCdif  = delta - deltaPAC(a,sig,T);
	highCdif = delta - deltaPAC(b,sig,T);
else
    a = 0.01;
    b = 1.0;
	lowCdif  = delta - deltaPAP(a,sig,T);
	highCdif = delta - deltaPAP(b,sig,T);
end

if lowCdif*highCdif > 0
	y = -1; % there's no solution
else
	for x=1:MaxIter
		midP = (a+b)/2;
		if delta > 0 
			midCdif = delta - deltaPAC(midP,sig,T);
		else
			midCdif = delta - deltaPAP(midP,sig,T);
		end
		if abs(midCdif)<Tol
			break
		else
			if midCdif>0
% 				a = midP;
                b = midP;
			else
% 				b = midP;
                a = midP;
			end
		end
	end
	y = midP;
end



In [ ]:
function y = BisecBSIV(PutCall,S,K,rf,T,a,b,MktPrice,Tol,MaxIter)

% Bisection algorithm
% PutCall = "P"ut or "C"all
% S = Spot price
% K = Strike price
% rf = Risk free rate
% T = Maturity
% a = Starting point lower value of vol
% b = Starting point upper value of vol
% MktPrice = Market price
% Tol = Tolerance
% MaxIter = maximum number of iterations

% Functions for the Black-Scholes call and put  
BSC = @(s,K,rf,v,T) (s.*normcdf((log(s./K) + (rf+v.^2./2).*T)./v./sqrt(T)) - K.*exp(-rf.*T).*normcdf((log(s./K) + (rf+v.^2./2).*T)./v./sqrt(T) - v.*sqrt(T)));
BSP = @(s,K,rf,v,T) (K.*exp(-rf.*T).*normcdf(-(log(s./K) + (rf+v.^2./2).*T)./v./sqrt(T) + v.*sqrt(T)) - s.*normcdf(-(log(s./K) + (rf+v.^2./2).*T)./v./sqrt(T)));
  
if strcmp(PutCall,'C')
	lowCdif  = MktPrice - BSC(S,K,rf,a,T);
	highCdif = MktPrice - BSC(S,K,rf,b,T);
else
	lowCdif  = MktPrice - BSP(S,K,rf,a,T);
	highCdif = MktPrice - BSP(S,K,rf,b,T);
end

if lowCdif*highCdif > 0
	y = -1;
else
	for x=1:MaxIter
		midP = (a+b)/2;
		if strcmp(PutCall,'C')
			midCdif = MktPrice - BSC(S,K,rf,midP,T);
		else
			midCdif = MktPrice - BSP(S,K,rf,midP,T);
		end
		if abs(midCdif)<Tol
			break
		else
			if midCdif>0
				a = midP;
			else
				b = midP;
			end
		end
	end
	y = midP;
end



In [ ]:
function y = BisecBlackFwdIV(PutCall,F,K,rf,T,a,b,MktPrice,Tol,MaxIter)

% Bisection algorithm
% PutCall = "P"ut or "C"all
% S = Spot price
% K = Strike price
% rf = Risk free rate
% T = Maturity
% a = Starting point lower value of vol
% b = Starting point upper value of vol
% MktPrice = Market price
% Tol = Tolerance
% MaxIter = maximum number of iterations


% Functions for the Black-Scholes call and put  
BSC = @(F,K,rf,v,T) (exp(-rf.*T).*(F.*H_ncdf((log(F./K) + (v.^2./2).*T)./v./sqrt(T)) - K.*H_ncdf((log(F./K) + (v.^2./2).*T)./v./sqrt(T) - v.*sqrt(T))));
BSP = @(F,K,rf,v,T) (exp(-rf.*T).*(K.*H_ncdf(-(log(F./K) + (v.^2./2).*T)./v./sqrt(T) + v.*sqrt(T)) - F.*H_ncdf(-(log(F./K) + (v.^2./2).*T)./v./sqrt(T))));
  
if strcmp(PutCall,'C')
	lowCdif  = MktPrice - BSC(F,K,rf,a,T);
	highCdif = MktPrice - BSC(F,K,rf,b,T);
else
	lowCdif  = MktPrice - BSP(F,K,rf,a,T);
	highCdif = MktPrice - BSP(F,K,rf,b,T);
end

if lowCdif*highCdif > 0
	y = -1;
else
	for x=1:MaxIter
		midP = (a+b)/2;
		if strcmp(PutCall,'C')
			midCdif = MktPrice - BSC(F,K,rf,midP,T);
		else
			midCdif = MktPrice - BSP(F,K,rf,midP,T);
		end
		if abs(midCdif)<Tol
			break
		else
			if midCdif>0
				a = midP;
			else
				b = midP;
			end
		end
	end
	y = midP;
end



In [ ]:
classdef BasisSwapHelper < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        
        rateType
        
        rate
        rateTenor
        
        gearing
        spread
        
        domesticIndex
        foreignIndex
        
        evaluationDate
        
        basisSwap
%         domesticDCurve
        discountCurve
        foreignDCurve
        
        earliestDate
        latestDate
        
    end
    
    methods
        % constructor
        function basisSwapHelper = BasisSwapHelper(params)
                if nargin > 0
                    basisSwapHelper.rateType = 'BasisSwap';
                    
                    basisSwapHelper.rate = params('rate');
                    basisSwapHelper.rateTenor = params('rateTenor');
                    
                    basisSwapHelper.gearing = params('domesticGearing');
                    basisSwapHelper.domesticIndex = params('domesticIndex');
                    basisSwapHelper.foreignIndex = params('foreignIndex');
                    
                    basisSwapHelper.evaluationDate = basisSwapHelper.domesticIndex.forwardCurve.asOfDate;
                    
%                     basisSwapHelper.domesticDCurve = params('domesticDCurve');
                    basisSwapHelper.discountCurve = params('discountCurve');
                    
                    % can be generalized later for dual curve input
                    
                    basisSwapHelper.foreignDCurve = basisSwapHelper.foreignIndex.forwardCurve;
                    
                    if basisSwapHelper.discountCurve.asOfDate ~= basisSwapHelper.domesticIndex.forwardCurve.asOfDate
                        error('forwardCurve asOfDate is different from discountCurve asOfDate')
                    end
                    
                    basisSwapHelper.basisSwap = MakeBasisSwap(basisSwapHelper);
                
                    out = InitializeDates(basisSwapHelper);

                    basisSwapHelper.earliestDate = out.earliestDate;
                    basisSwapHelper.latestDate = out.latestDate;
                end
        end
        
        function out = MakeBasisSwap(basisSwapHelper)
            domesticIndex = basisSwapHelper.domesticIndex;
            foreignIndex = basisSwapHelper.foreignIndex;
            
            domesticCalendar = domesticIndex.fixingCalendar;
            foreignCalendar = foreignIndex.fixingCalendar;
            
            jointCalendar = JointCalendar({domesticCalendar,foreignCalendar});
            
            fixingDays = max(domesticIndex.fixingDays,foreignIndex.fixingDays);
            
            referenceDate = domesticIndex.forwardCurve.asOfDate;
            
            startDate = jointCalendar.Advance(referenceDate,'Following',...
                                                fixingDays,'day',false);
            
            endDate = AddTenor(startDate,basisSwapHelper.rateTenor);
            % fixed & floating schedule 
            domesticSchedule = Schedule(startDate,endDate,domesticIndex.tenor ,jointCalendar,...
                                    domesticIndex.convention, 'Backward',false); 
            
            foreignSchedule = Schedule(startDate,endDate,foreignIndex.tenor,jointCalendar,...
                                    foreignIndex.convention, 'Backward',false);
             
             type = 'Payer';
%              nominal = 10000.0;
             domesticNominal = 1.0;
             domesticGearing = basisSwapHelper.gearing;
             domesticSpread = 0.0;
             domesticDayCounter = domesticIndex.dayCounter;
             
             foreignNominal = 1.0;
             foreignGearing = 1.0;
             foreignSpread = 0.0;
             foreignDayCounter = foreignIndex.dayCounter;
             
             notionalExchangeType = 'Both';
             
             basisSwap = BasisSwap(type,domesticNominal,domesticSchedule,domesticGearing,domesticSpread,domesticDayCounter, ...
                                   domesticIndex,... 
                                   foreignNominal,foreignSchedule,foreignGearing,foreignSpread,foreignDayCounter,...
                                   foreignIndex,...
                                   notionalExchangeType);
                                  
             out = basisSwap;
        end
        
        function out = InitializeDates(basisSwapHelper)
            
            %earliestDate
            earliestDate = basisSwapHelper.basisSwap.StartDate();
            latestDate = basisSwapHelper.basisSwap.EndDate();
            
            % need to be implemented ??
            % check later !!
            
%             floatEndValueDate = basisSwapHelper.vanillaSwap.floatingLeg.couponRates{end}.fixingEndDate; 
%             % if the last floating Index fixingEndDate is after
%             % latestDate(last accrualEndDate then we change
%             % the lastestDate into floatEndValueDate
%             
%             if DateDiff(floatEndValueDate,latestDate) > 0
%                 latestDate = floatEndValueDate;
%             end
            
            out.earliestDate = earliestDate;
            out.latestDate = latestDate;
           
        end
        
        function out = ImpliedQuote(basisSwapHelper)
            outInfo = basisSwapHelper.basisSwap.Calculate(basisSwapHelper.discountCurve,basisSwapHelper.foreignDCurve);
            out = outInfo.fairDomesticSpread;
            
        end
        
        % bootstrapping
        function out = QuoteError(basisSwapHelper)
            out = basisSwapHelper.rate - basisSwapHelper.ImpliedQuote();
        end
        
        function out = BootStrapError(basisSwapHelper,guess,idx)
            basisSwapHelper.Update(idx,guess);
            out = basisSwapHelper.QuoteError();
        end
        
        function Update(basisSwapHelper,idx,guess)
             % forwardCurve update
%             forwardRawData = basisSwapHelper.iborIndex.forwardCurve.params('rawData');
%             forwardRawData(idx,2) = guess;
%             basisSwapHelper.iborIndex.forwardCurve.params('rawData')= forwardRawData;
            
            % for singleCurve framework this is redundant
            
            % discountCurve update
            discountRawData = basisSwapHelper.discountCurve.params('rawData');
            discountRawData(idx,2) = guess;
            basisSwapHelper.discountCurve.params('rawData')= discountRawData;
            
        end
        
        
    end
    
end



In [ ]:
classdef BasisSwap < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        type
        domesticNominal
        domesticSchedule
        domesticGearing
        domesticSpread
        domesticDayCounter
        domesticIndex
        
        foreignNominal
        foreignSchedule
        foreignGearing
        foreignSpread
        foreignDayCounter
        foreignIndex
        notionalExchangeType
        
        domesticLeg
        foreignLeg
        
        NPV;
        fairDomesticSpread;
       
    end
    
    methods
        % constructor
        function basisSwap = BasisSwap(type,domesticNominal,domesticSchedule,domesticGearing,domesticSpread,domesticDayCounter,...
                                      domesticIndex,...
                                      foreignNominal,foreignSchedule,foreignGearing,foreignSpread,foreignDayCounter,...
                                      foreignIndex,...
                                      notionalExchangeType)
            
            basisSwap.type = type;
            basisSwap.domesticNominal = domesticNominal;
            basisSwap.domesticSchedule = domesticSchedule;
            basisSwap.domesticGearing = domesticGearing;
            basisSwap.domesticSpread = domesticSpread;
            basisSwap.domesticDayCounter = domesticDayCounter;
            basisSwap.domesticIndex = domesticIndex;
            
            basisSwap.foreignNominal = foreignNominal;
            basisSwap.foreignSchedule = foreignSchedule;
            basisSwap.foreignGearing = foreignGearing;
            basisSwap.foreignSpread = foreignSpread;
            basisSwap.foreignDayCounter = foreignDayCounter;
            basisSwap.foreignIndex = foreignIndex;
            
            basisSwap.notionalExchangeType = notionalExchangeType;
            
            basisSwap.domesticLeg = FixedRateLeg(domesticSchedule,domesticNominal,1.0,domesticDayCounter,domesticSchedule.convention);
                                    
            if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Initial')
                couponRates = basisSwap.domesticLeg.couponRates;
                tempRates = cell(length(couponRates)+1,1);
                tempRates{1} = SimpleCashflow(-domesticNominal,domesticSchedule.startDate);
                for i=2:length(tempRates)
                    tempRates{i} = couponRates{i-1};
                end
                basisSwap.domesticLeg.couponRates = tempRates;
            end
            
            if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Final')
                basisSwap.domesticLeg.couponRates{end+1} = SimpleCashflow(domesticNominal,domesticSchedule.endDate);
            end
            
            
            basisSwap.foreignLeg = FloatingRateLeg(foreignSchedule,foreignIndex,foreignGearing,foreignSpread,foreignNominal,...
                                      foreignDayCounter,foreignSchedule.convention);
                                  
            if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Initial')
                couponRates = basisSwap.foreignLeg.couponRates;
                tempRates = cell(length(couponRates)+1,1);
                tempRates{1} = SimpleCashflow(-foreignNominal,foreignSchedule.startDate);
                for i=2:length(tempRates)
                    tempRates{i} = couponRates{i-1};
                end
                basisSwap.foreignLeg.couponRates = tempRates;
            end
            
            if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Final')
                basisSwap.foreignLeg.couponRates{end+1} = SimpleCashflow(foreignNominal,foreignSchedule.endDate);
            end
            
        end
        
        function out = StartDate(basisSwap)
            d1 = basisSwap.domesticLeg.StartDate();
            d2 = basisSwap.foreignLeg.StartDate();
            
            if DateDiff(d1,d2) > 0
                out = d2;
            else
                out = d1;
            end
        end
        
        function out = EndDate(basisSwap)
            d1 = basisSwap.domesticLeg.EndDate();
            d2 = basisSwap.foreignLeg.EndDate();
            
            if DateDiff(d1,d2) > 0
                out = d1;
            else
                out = d2;
            end
        end
        
        function out = Calculate(basisSwap,domesticDCurve,foreignDCurve)
            % to be updated
            % very weird samsung basisSwap pricing logic :(
            % talk to CSY in the future
            
            domesticNPV = basisSwap.domesticLeg.AdjustedNPV(domesticDCurve);
            domesticBPS = basisSwap.domesticLeg.AdjustedBPS(domesticDCurve);
%             spreadNPV = basisSwap.floatingSpread * domesticBPS;
            foreignNPV = basisSwap.foreignLeg.NPV(foreignDCurve);
%             foreignBPS = basisSwap.fixedLeg.BPS(discountCurve);
            
            if strcmp(basisSwap.type,'Payer')
                out.NPV = domesticNPV - domesticBPS - foreignNPV;
                out.fairDomesticSpread =  -1.0*(out.NPV)/domesticBPS;
            else
                out.NPV = -1.0*(domesticNPV + domesticBPS - foreignNPV);
                out.fairDomesticSpread =  1.0*(out.NPV)/domesticBPS;
            end
            
        end
    end
    
end



In [ ]:
function y=backshift(day,x)
% y=backshift(day,x)
assert(day>=0);
y=[NaN(day,size(x,2), size(x, 3));x(1:end-day,:, :)];


In [ ]:
classdef AFVSimulation < GammaKernel
    
    methods
        function out = psiM(obj,psi,ev,w)
            beta2 = 2.0./psi-1+sqrt(2./psi).*sqrt(abs(2./psi-1));
            alpha = ev./(1+beta2);
            vf = alpha.*(sqrt(abs(beta2))+w).^2;
            out = vf;
        end
        
        function out = psiP(obj,psi,ev,u)
            p = 2.0./(1+psi);
            gam = ev./2.0.*(1+psi);
            vf = -(u<p).*gam.*log(u./p);
            out = vf;
        end

        function out = RSQESim(obj, params, xi,paths, steps, expiry)
            try
                eta = params.eta;
                lam = params.lam;
                H = params.al - 1/2;
                rho = params.rho;
                rho2m1 = sqrt(1 - rho * rho);
                eps0 = 1e-10;
                %     W = matrix(rnorm(steps * paths), nrow = steps, ncol = paths);
                rng('default');
                rng(0);
                
                W = randn(paths/2,steps);W=[W;-W];W=W';
                Wperp = randn(paths/2,steps);Wperp=[Wperp;-Wperp];Wperp=Wperp';
                Un = randn(paths/2,steps);Un=[Un;-Un];Un=Un';
                U = normcdf(Un);
                
                G00p = @(t) (arrayfun(@(t) obj.G00(params,t),t));
    
                dt = expiry/steps;
                sqrtDt = sqrt(dt);
                tj = (1:steps) * dt;
                xij = xi(tj);
                G00del = obj.G00(params,dt);
                

                G00j = [0, G00p(tj)];

                bstar = sqrt(diff(G00j)./dt);

                bstar1 = bstar(1); % bstar is average g over an interval
                u = zeros(steps,paths);
                v = ones(1,paths).* xi(0);
                xihat = ones(1,paths).*xij(1);
                x = zeros(1,paths);
                y = zeros(1,paths);
                w = zeros(1,paths);
                vf = zeros(1,paths);
                for j =1:steps 
                    varv = eta.^2 .* (xihat + 2.* H.* v)/(1+2.*H).* G00del;
                    psi =varv./xihat.^2;
                    vf = (psi < 3/2).*obj.psiM(psi, xihat,W(j,:))+(psi >= 3/2).*obj.psiP(psi, xihat, U(j,:));

                    u(j,:) = vf - xihat;
                    dw = (v + vf)./2 .* dt;
                    w  = w + dw;
                    dy = u(j,:)./(eta .* bstar1);
                    y = y + dy;
                    x = x - dw./2 + sqrt(dw) .* rho2m1 .* Wperp(j,:) + rho .* dy;

                    if (j < steps)
                        btilde = flip(bstar(2:(j+1)));

                        xihat = xij(j + 1) + (btilde * u(1:j,:))./bstar1;
                    end

                    idx11 = find(xihat <= eps0);
                    if ~isempty(idx11)
                        xihat(idx11) = eps0;
                    end

                    v = vf;
                end

                resSim.v = v;
                resSim.x = x;
                resSim.y = y;
                resSim.w = w;

                out = resSim;
            catch ME
                aaaa = 1.0;
                rethrow(ME)
            end
        end
        
        function out = RSQESimHist(obj, params, xi,paths, steps, expiry)
            try
                eta = params.eta;
                lam = params.lam;
                H = params.al - 1/2;
                rho = params.rho;
                rho2m1 = sqrt(1 - rho * rho);
                eps0 = 1e-10;
                %     W = matrix(rnorm(steps * paths), nrow = steps, ncol = paths);
                rng('default');
                rng(0);
                
                W = randn(paths/2,steps);W=[W;-W];W=W';
                Wperp = randn(paths/2,steps);Wperp=[Wperp;-Wperp];Wperp=Wperp';
                Un = randn(paths/2,steps);Un=[Un;-Un];Un=Un';
                U = normcdf(Un);
                
                G00p = @(t) (arrayfun(@(t) obj.G00(params,t),t));
    
                dt = expiry/steps;
                sqrtDt = sqrt(dt);
                tj = (1:steps) * dt;
                xij = xi(tj);
                G00del = obj.G00(params,dt);
                

                G00j = [0, G00p(tj)];

                bstar = sqrt(diff(G00j)./dt);

                bstar1 = bstar(1); % bstar is average g over an interval
                u = zeros(steps,paths);
                v = ones(1,paths).* xi(0);
                xihat = ones(1,paths).*xij(1);
                x = zeros(1,paths);
                y = zeros(1,paths);
                w = zeros(1,paths);
                vf = zeros(1,paths);
                
                % log S(t)/F(t) 's simulation step for all paths
                xhist = zeros(steps,paths);
                
                for j =1:steps 
                    varv = eta.^2 .* (xihat + 2.* H.* v)/(1+2.*H).* G00del;
                    psi =varv./xihat.^2;
                    vf = (psi < 3/2).*obj.psiM(psi, xihat,W(j,:))+(psi >= 3/2).*obj.psiP(psi, xihat, U(j,:));

                    u(j,:) = vf - xihat;
                    dw = (v + vf)./2 .* dt;
                    w  = w + dw;
                    dy = u(j,:)./(eta .* bstar1);
                    y = y + dy;
                    x = x - dw./2 + sqrt(dw) .* rho2m1 .* Wperp(j,:) + rho .* dy;

                    if (j < steps)
                        btilde = flip(bstar(2:(j+1)));

                        xihat = xij(j + 1) + (btilde * u(1:j,:))./bstar1;
                    end

                    idx11 = find(xihat <= eps0);
                    if ~isempty(idx11)
                        xihat(idx11) = eps0;
                    end

                    v = vf;
                    
                    xhist(j,:) = x;
                end

                resSim.v = v;
                resSim.x = x;
                resSim.y = y;
                resSim.w = w;
                
                resSim.xhist = xhist;
                out = resSim;
            catch ME
                aaaa = 1.0;
                rethrow(ME)
            end
        end
        
        function out = HQESim(obj, params, xi,paths, steps, expiry)
            try
                eta = params.eta;
                lam = params.lam;
                H = params.al - 1/2;
                rho = params.rho;
                rho2m1 = sqrt(1 - rho * rho);
                eps0 = 1e-10;
                %     W = matrix(rnorm(steps * paths), nrow = steps, ncol = paths);
                
                rng('default');
                rng(0);
                
                W = randn(paths/2,steps);W=[W;-W];W=W';
                Wperp = randn(paths/2,steps);Wperp=[Wperp;-Wperp];Wperp=Wperp';
                Z = randn(paths/2,steps);Z=[Z;-Z];Z=Z';
                Un = randn(paths/2,steps);Un=[Un;-Un];Un=Un';
                U = normcdf(Un);
                Uperpn = randn(paths/2,steps);Uperpn=[Uperpn;-Uperpn];Uperpn=Uperpn';
                Uperp = normcdf(Uperpn);
                
                G00p = @(t) (arrayfun(@(t) obj.G00(params,t),t));
    
                dt = expiry/steps;
                sqrtDt = sqrt(dt);
                tj = (1:steps) * dt;
                xij = xi(tj);
%                 G00del = obj.G00(params,dt);
                G0del = eta.*obj.G0(params,dt);
                G1del  = eta.*obj.G1(params,dt);
                G00del = eta.^2.*obj.G00(params,dt);
                G11del = eta.^2.*obj.G11(params,dt);
                G01del = eta.^2.*obj.G01(params,dt);
                G00j   = eta.^2.*[0, G00p(tj)];
                bstar  = sqrt(diff(G00j)./dt);
                bstar1 = bstar(1); % # bstar is average g over an interval
                rho_vchi = G0del/sqrt(G00del*dt);
                beta_vchi = G0del/dt;

                u = zeros(steps,paths);
                chi = zeros(steps,paths);
                v = ones(1,paths).* xi(0);
                xihat = ones(1,paths).*xij(1);
                
                x = zeros(1,paths);
                y = zeros(1,paths);
                w = zeros(1,paths);
                vf = zeros(1,paths);
                
                for j =1:steps 
                    xibar = (xihat + 2 .* H.* v)./(1 + 2.* H);
                    var_chi = xibar.* dt;
                    var_eps = xibar.* G00del.*(1-rho_vchi.^2);
                    psi_chi = 4.*G00del.*rho_vchi.^2./xibar;
                    psi_eps = 4.*G00del.*(1-rho_vchi.^2)./xibar;

                    z_chi = (psi_chi < 3/2).*obj.psiM(psi_chi, xihat./2,W(j,:))+(psi_chi >= 3/2).*obj.psiP(psi_chi, xihat./2, U(j,:));
                    z_eps = (psi_eps < 3/2).*obj.psiM(psi_eps, xihat./2,Wperp(j,:))+(psi_eps >= 3/2).*obj.psiP(psi_eps, xihat./2, Uperp(j,:));
                    
                    chi(j,:) = (z_chi-xihat./2)./beta_vchi;
                    eps = z_eps-xihat./2;
                    u(j,:) = beta_vchi.*chi(j,:) + eps;
                    vf = xihat + u(j,:);
                    
                    idx11 = find(vf <= eps0);
                    if ~isempty(idx11)
                        vf(idx11) = eps0;
                    end
                    
                    dw = (v + vf)/2 .* dt;
                    w  =  w + dw; 
                    y  =  y + chi(j,:);
                    x  =  x - dw./2 + sqrt(dw) .*rho2m1 .* Z(j,:) + rho.* chi(j,:);
                    
                    if (j < steps)
                        btilde = flip(bstar(2:(j+1)));

                        xihat = xij(j + 1) + (btilde * chi(1:j,:));
                    end

                    v = vf;

                end

                resSim.v = v;
                resSim.x = x;
                resSim.y = y;
                resSim.w = w;

                out = resSim;
                
            catch ME
                aaaa = 1.0;
                rethrow(ME)
            end
        end
        
        function out = HQESimHist(obj, params, xi,paths, steps, expiry)
            try
                eta = params.eta;
                lam = params.lam;
                H = params.al - 1/2;
                rho = params.rho;
                rho2m1 = sqrt(1 - rho * rho);
                eps0 = 1e-10;
                %     W = matrix(rnorm(steps * paths), nrow = steps, ncol = paths);
                
                rng('default');
                rng(0);
                
                W = randn(paths/2,steps);W=[W;-W];W=W';
                Wperp = randn(paths/2,steps);Wperp=[Wperp;-Wperp];Wperp=Wperp';
                Z = randn(paths/2,steps);Z=[Z;-Z];Z=Z';
                Un = randn(paths/2,steps);Un=[Un;-Un];Un=Un';
                U = normcdf(Un);
                Uperpn = randn(paths/2,steps);Uperpn=[Uperpn;-Uperpn];Uperpn=Uperpn';
                Uperp = normcdf(Uperpn);
                
                % log S(t)/F(t) 's simulation step for all paths
                xhist = zeros(steps,paths);
                
                G00p = @(t) (arrayfun(@(t) obj.G00(params,t),t));
    
                dt = expiry/steps;
                sqrtDt = sqrt(dt);
                tj = (1:steps) * dt;
                xij = xi(tj);
%                 G00del = obj.G00(params,dt);
                G0del = eta.*obj.G0(params,dt);
                G1del  = eta.*obj.G1(params,dt);
                G00del = eta.^2.*obj.G00(params,dt);
                G11del = eta.^2.*obj.G11(params,dt);
                G01del = eta.^2.*obj.G01(params,dt);
                G00j   = eta.^2.*[0, G00p(tj)];
                bstar  = sqrt(diff(G00j)./dt);
                bstar1 = bstar(1); % # bstar is average g over an interval
                rho_vchi = G0del/sqrt(G00del*dt);
                beta_vchi = G0del/dt;

                u = zeros(steps,paths);
                chi = zeros(steps,paths);
                v = ones(1,paths).* xi(0);
                xihat = ones(1,paths).*xij(1);
                
                x = zeros(1,paths);
                y = zeros(1,paths);
                w = zeros(1,paths);
                vf = zeros(1,paths);
                
                for j =1:steps 
                    xibar = (xihat + 2 .* H.* v)./(1 + 2.* H);
                    var_chi = xibar.* dt;
                    var_eps = xibar.* G00del.*(1-rho_vchi.^2);
                    psi_chi = 4.*G00del.*rho_vchi.^2./xibar;
                    psi_eps = 4.*G00del.*(1-rho_vchi.^2)./xibar;

                    z_chi = (psi_chi < 3/2).*obj.psiM(psi_chi, xihat./2,W(j,:))+(psi_chi >= 3/2).*obj.psiP(psi_chi, xihat./2, U(j,:));
                    z_eps = (psi_eps < 3/2).*obj.psiM(psi_eps, xihat./2,Wperp(j,:))+(psi_eps >= 3/2).*obj.psiP(psi_eps, xihat./2, Uperp(j,:));
                    
                    chi(j,:) = (z_chi-xihat./2)./beta_vchi;
                    eps = z_eps-xihat./2;
                    u(j,:) = beta_vchi.*chi(j,:) + eps;
                    vf = xihat + u(j,:);
                    
                    idx11 = find(vf <= eps0);
                    if ~isempty(idx11)
                        vf(idx11) = eps0;
                    end
                    
                    dw = (v + vf)/2 .* dt;
                    w  =  w + dw; 
                    y  =  y + chi(j,:);
                    x  =  x - dw./2 + sqrt(dw) .*rho2m1 .* Z(j,:) + rho.* chi(j,:);
                    
                    if (j < steps)
                        btilde = flip(bstar(2:(j+1)));

                        xihat = xij(j + 1) + (btilde * chi(1:j,:));
                    end

                    v = vf;
                    
                    xhist(j,:) = x;

                end

                resSim.v = v;
                resSim.x = x;
                resSim.y = y;
                resSim.w = w;
                
                resSim.xhist = xhist;

                out = resSim;
                
            catch ME
                aaaa = 1.0;
                rethrow(ME)
            end
        end
        
    end
end



In [ ]:
%
% Calculating sensitivites of local vol. surface of a EU call using adjoint
% technique
% Use Euler-Maruyama method and Pathwise Sensitivity Method to
% estimate sensitivity of call option to spline points of local vol.
% surface
% Test problem: dS = r*S dt + sig*S dW
%
function [LocalSens,LocalSensConf]=adjoint_mode

    % problem parameters
    r = 0.05;
    sig0 = 0.5;
    T = 1;
    S0 = 100;
    K = 110;
    %parameters for local vol grid
    J = 20;
    I = 20; %I < N
    % Monte Carlo simulation parameters
    M = 1e+6; % total number of Monte Carlo paths
    M2 = 1e+4; % number of paths at a time
    N = 100;
    h = T/N;
    % local vol surface grid setup
    Smin = 0.8*S0;
    Smax = 1.2*S0;
    dS = (Smax - Smin)/J;
    dT = T/I;
    %set up our testing local vol surface
    LocalVolSurf = ones(I+1,J+1)*sig0;
    
    for i=1:I+1
        LocalVolSurf(i,:) = LocalVolSurf(i,:)*((I+1)*((i^(-1)+i^(-0.5))/2)/(I+1));
    end
    
    %set up variable for storing sensitivities
    LocalSens = zeros(I+1,J+1); %sensitivities
    LocalSensConf = zeros(I+1,J+1); %confidence interval for MC sampling
    %iIndex - integer from interval [0,I]
    %beta - time interpolation parameter
    iIndex = floor([1:N]*h./dT);
    beta = ([1:N].*h-iIndex([1:N]).*dT)./dT;
    
    iIndex = [0,iIndex];
    beta = [0,beta];
    %setting up interpolation over time
    sigJ = zeros(N+1,J+1);
    for n=0:N
        sigJ(n+1,:) = LocalVolSurf(1+iIndex(1+n),:) + beta(1+n) * (...
        LocalVolSurf(min(1+iIndex(1+n)+1,I+1),:) - LocalVolSurf(1+iIndex(1+n),:));
    end
    
    %main MC sampling loop
    for m = 1:M2:M
        m2 = min(M2,M-m+1);
        S = S0*ones(1,m2);
        D = zeros(N,m2);
        B = zeros(N,m2);
        alpha = zeros(N+1,m2);
        jIndex = zeros(N+1,m2);
        A = zeros(N+1,m2);
        for n = 1:N
            dW = sqrt(h)*randn(1,m2);
            %jIndex - integer from interval [0,J] - indicates which block S is in
            jIndex(n,:) = max( min(J,floor(real((S-Smin)./dS))) ,0);
            %alpha - stock price interpolation parameter
            alpha(n,:) = (S - (Smin + jIndex(n,:).*dS) )./dS;
            %linear interpolation in stock price
            sig = sigJ(n,1+jIndex(n,:)) + ...
            alpha(n,:).*(sigJ(n,min(J+1,1+jIndex(n,:)+1)) - sigJ(n,1+jIndex(n,:)) );
            %storing necessary variables
            D(n,:) = 1+r*h+sig.*dW;
            A(n,:) = (1/dS).*(sigJ(n,min(J+1,1+jIndex(n,:)+1)) - ...
            sigJ(n,1+jIndex(n,:)) );
            B(n,:) = S.*dW;
            %timestep
            S = S.*D(n,:);
        end

        jIndex(N+1,:) = max( min(J,floor(real((S-Smin)./dS))) ,0);
        alpha(N+1,:) = (S - (Smin + jIndex(N+1,:).*dS) )./dS;

        %--------------------------ADJOINT METHOD-----------------------------%
        %setting Sbar(N) - recursion for calculating dP/dS0
        Sbar = zeros(1,m2);
        for i = 1:m2
            if(S(i)>K)
                Sbar(i) = exp(-r*T);
            else
                Sbar(i) = 0;
            end
        end

        sigbarJ = zeros(N+1,J+1,m2);
        sigbarIJ = zeros(I+1,J+1,m2);
        for n=0:(N-1)
            %recurrence for sigbar, Sbar
            sigbar = Sbar.*B(N-n,:);
            Sbar = (D(N-n,:)+B(N-n,:).*A(N-n,:)).*Sbar;
            %sigbarJs
            for k=1:m2
                j = 1 + jIndex(N-n,k);%in [1,J+1]
                if(j<J+1)
                    sigbarJ(N-n,j,k)= (1-alpha(N-n,k))*sigbar(k);
                    sigbarJ(N-n,j+1,k) = alpha(N-n,k)*sigbar(k);
                else
                    sigbarJ(N-n,j,k) = sigbar(k);
                end
            end

            %sigbarIJs
            for k=1:m2
                i = 1 + iIndex(N-n);
                j = 1+jIndex(N-n,k);
                sigbarIJ(i,j,k) = sigbarIJ(i,j,k) + (1-beta(N-n)).*sigbarJ(N-n,j,k);
                if(j<J+1)
                    sigbarIJ(i,j+1,k) = ...
                    sigbarIJ(i,j+1,k) + (1-beta(N-n)).*sigbarJ(N-n,j+1,k);
                    if(i<I+1)
                        sigbarIJ(i+1,j+1,k) = ...
                        sigbarIJ(i+1,j+1,k) + beta(N-n).*sigbarJ(N-n,j+1,k);
                    end
                end

                if(i<I+1)
                    sigbarIJ(i+1,j,k) = ...
                    sigbarIJ(i+1,j,k) + beta(N-n).*sigbarJ(N-n,j,k);
                end
            end
        end
        
        LocalSens = LocalSens + sum(sigbarIJ,3);
        LocalSensConf = LocalSensConf + sum(sigbarIJ.^2,3);
        
    end
    
    LocalSens = LocalSens./M;
    LocalSensConf = (LocalSensConf./M - LocalSens.^2);
    LocalSensConf = 3*sqrt((M/(M-1)) .* LocalSensConf)./sqrt(M);

end


In [ ]:
import pandas as pd
import requests
from lxml import html
from tqdm import tqdm

In [ ]:
# 삼성전자
sample_code = '005930'

In [ ]:
# parsing URL
# 우리 컴퓨터 -> [접속] -> 에프앤가이드(Data Source) -> [크롤링/웹 스크래핑] -> 우리 컴퓨터
# client -> request -> [Server] -> response -> client

SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'


In [ ]:
# object -> 데이터 덩어리(추상화)

snap_url = SNAP_URL.format(sample_code)
snap_content = requests.get(snap_url).content # 문자열 binary
snap_tree = html.fromstring(snap_content) # 객체(object)
per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
per = float(per)

In [ ]:
per

8.15

In [ ]:
ratio_url = RATIO_URL.format(sample_code)
ratio_content = requests.get(ratio_url).content
ratio_tree = html.fromstring(ratio_content)
debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
debt_ratio = float(debt_ratio)

In [ ]:
# stockMkt => KOSPI
# kosdaqMkt => KOSDAQ
# konexMkt => KONEX

In [ ]:
def get_stock_list(market):
    market_code = ''
    if market == 'kospi':
        market_code = 'stockMkt'
    elif market == 'kosdaq':
        market_code = 'kosdaqMkt'
    elif market == 'konex':
        market_code = 'konexMkt'
    kind_url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&pageIndex=1&currentPageSize=3000&comAbbrv=&beginIndex=&orderMode=3&orderStat=D&isurCd=&repIsuSrtCd=&searchCodeType=&marketType={}&searchType=13&industry=&fiscalYearEnd=all&comAbbrvTmp=&location=all'.format(market_code)                                                                 

    return pd.read_html(kind_url, converters={'종목코드':lambda x: str(x)})[0]


In [ ]:
def converter(x):
    return str(x)

# 람다 함수 == 무명(anonymous) 함수 == 일회용 함수
lambda x: str(x)

<function __main__.<lambda>(x)>

In [ ]:
kospi_df = get_stock_list('kospi')
print(kospi_df.shape)

(829, 9)


In [ ]:
kosdaq_df = get_stock_list('kosdaq')
print(kosdaq_df.shape)

(1632, 9)


In [ ]:
# merge -> SQL Join
# append | kospi_df.append([kosdaq_df])
# concatenate(합치다)
stock_list_df = pd.concat([kospi_df, kosdaq_df] )

In [ ]:
print(stock_list_df.shape)

(2461, 9)


In [ ]:
# stock_list_df['종목코드'].dropna()
stock_list_df = stock_list_df[stock_list_df['종목코드'].notnull()]

In [ ]:
stock_list_df = stock_list_df[~stock_list_df['회사명'].str.contains('스팩|리츠')]
print(stock_list_df.shape)

(2364, 9)


In [ ]:
# list comprehension
stock_list_df.index = [x for x in range(len(stock_list_df))]

In [ ]:
stock_list_df.to_csv('kospi_kosdaq_stock_list.csv', encoding='utf-8', index=True)

In [ ]:
code_list = stock_list_df['종목코드']
code_list

0       100090
1       453340
2       452260
3       450140
4       377740
         ...  
2359    013030
2360    019550
2361    019570
2362    019590
2363    006920
Name: 종목코드, Length: 2364, dtype: object

In [ ]:
sample_df = pd.DataFrame(
    {'005930':['삼성전자', 1, 2], '035720':['카카오', 1, 2], '015720':['카카오', 1, 2], '025720':['카카오', 1, 2]}
).transpose()



In [ ]:
sample_df.columns = ['name', 'PER', 'Debt_ratio']

In [ ]:
sample_df

,name,PER,Debt_ratio
005930,삼성전자,1,2
035720,카카오,1,2
015720,카카오,1,2
025720,카카오,1,2


In [ ]:
def FinanceInfoCrawler(li, df):
    result_dict = {}
    error_codes = []
    
    for code in tqdm(li):
        try:
            # Parsing URL setting
            SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
            RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

            # company name
            company_name = df[df['종목코드'] == code]['회사명'].values[0]

            # Get PER
            snap_url = SNAP_URL.format(code)
            snap_content = requests.get(snap_url).content
            snap_tree = html.fromstring(snap_content)
            per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
            per = float(per)

            # Get Debt ratio
            ratio_url = RATIO_URL.format(code)
            ratio_content = requests.get(ratio_url).content
            ratio_tree = html.fromstring(ratio_content)
            debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
            debt_ratio = float(debt_ratio)
            
            result_dict[code] = [company_name, per, debt_ratio]
            
        except (TypeError, IndexError, AttributeError, ValueError):
            pass
#             print(code)
            error_codes.append(code)
    
    # convert dict to DataFrame
    result_df = pd.DataFrame(result_dict)
    
    # transpose DataFrame
    result_df = result_df.transpose()
    
    # Setting column names
    result_df.columns = ['Name', 'PER', 'Debt_ratio']
    
    return result_df, error_codes

In [ ]:
crawling_result_df = FinanceInfoCrawler(code_list[:50],stock_list_df)

100%|██████████| 50/50 [00:46<00:00,  1.07it/s]


In [ ]:
print(crawling_result_df[0].shape)
crawling_result_df[0].head()

(39, 3)


,Name,PER,Debt_ratio
100090,SK오션플랜트,38.11,132.0
377740,바이오노트,1.89,9.8
446070,유니드비티플러스,104.18,11.2
108320,LX세미콘,7.5,35.7
126720,수산인더스트리,6.3,24.7


In [ ]:
# original data
# crawling_result_df

# copy data
copy_df = crawling_result_df[0].copy()

In [ ]:
# PER 10 이하
# 부채비율 50 이하
# (상위) 20개 종목

final_result_df = copy_df[
    (copy_df['PER'] <= 10)&(copy_df['Debt_ratio'] <= 50)&(copy_df['PER'] > 0)
].sort_values(
    by='PER', ascending=True
).iloc[:20]


In [ ]:
import datetime

# 시간까지 포함한 날짜
now = datetime.datetime.now()

final_result_df.to_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

In [ ]:
pd.read_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

,Unnamed: 0,Name,PER,Debt_ratio
0,377740,바이오노트,1.89,9.8
1,137310,에스디바이오센서,2.39,10.7
2,383800,LX홀딩스,3.90,1.5
3,353200,대덕전자,6.16,39.4
4,363280,티와이홀딩스,6.17,47.6
5,126720,수산인더스트리,6.30,24.7
6,108320,LX세미콘,7.50,35.7
